In [1]:
import os
import sys as sys

import itertools
from itertools import cycle,groupby
import re
from re import search

import regex
import xlrd
#import xlwt
# To install run: !conda install xlwt or !pip install xlwt or similar in your environment

import glob
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.ticker as plticker
import seaborn
import numpy as np
from numpy import linspace
from copy import deepcopy
import traceback
import warnings
import argparse
import subprocess
import ftplib as ftp

#Don't need:
# import scipy.stats
# import scipy.cluster
# from scipy import interp
# from scipy.stats import norm
# from scipy.stats.kde import gaussian_kde
# import sklearn

import ete3 as ete3
from ete3 import Tree, PhyloTree #, TreeStyle, TextFace, add_face_to_node #Looks like these are deprecated??

import Bio as Bio
from Bio import SeqIO, AlignIO
from Bio import Phylo, _utils
from Bio.Phylo import BaseTree

#!conda install IO

In [2]:
!pwd

/nobackup1c/users/payette/ABGDtree5


In [3]:
os.chdir('/nobackup1c/users/payette/ABGDtree5/sequences/')
!pwd

/nobackup1c/users/payette/ABGDtree5/sequences


In [4]:
def fasta_iter(fasta_name):
    fasta_nam = open(fasta_name)
    fasta_s = (x[1] for x in groupby(fasta_nam, lambda line: line[0] == ">"))
    for header in fasta_s:
                                      # drop the ">" #turned off
        headerStr = header.__next__() #[1:].strip()
        # join all sequence lines to one.
        seq = "".join(s.strip() for s in fasta_s.__next__())
        yield (headerStr, seq)

In [5]:
def fast_fasta(fasta_file,protein_name):
    for header, seq in fasta_iter(fasta_file):
        #print(header)
        if header.find(protein_name) != -1:
            print(header,seq)
            #print(seq)
            return(header,seq)
        else:
            continue  

In [6]:
class cd:
    """
    Context manager for changing the current working directory
    """
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [7]:
def checkGenome(file_index):    
    num_complete = 0    
    for protein_name in large_subunits+small_subunits:
        num_complete = 0
        print(protein_name)
        for header, seq in fasta_iter(file_index):
            if header.find(protein_name) != -1:
                print(header,seq) #Turn this on to get specific.
                num_complete = num_complete+1
            else:
                continue
            print(num_complete)

In [8]:
#Manually import data for protein identification from .faa RefSeq / Genbank files
#Please note this is not a automatic process and needs by-hand adjustment

large_subunits=['50S ribosomal protein L1 [','50S ribosomal protein L2 [','50S ribosomal protein L3 [',
                '50S ribosomal protein L4 [','50S ribosomal protein L5 [','50S ribosomal protein L6 [',
                '50S ribosomal protein L10 [','50S ribosomal protein L13 [','50S ribosomal protein L14 [',
                '50S ribosomal protein L15 [','50S ribosomal protein L18 [','50S ribosomal protein L22 [',
                 '50S ribosomal protein L23 [','50S ribosomal protein L24 [','50S ribosomal protein L29 [']
print(len(large_subunits))

15


In [9]:
small_subunits=['30S ribosomal protein S2 [','30S ribosomal protein S3 [','30S ribosomal protein S4 [',
                '30S ribosomal protein S5 [','30S ribosomal protein S7 [','30S ribosomal protein S8 [','30S ribosomal protein S9 [',
                '30S ribosomal protein S10 [',
 '30S ribosomal protein S11 [','30S ribosomal protein S12 [','30S ribosomal protein S13 [','30S ribosomal protein S14 [',
 '30S ribosomal protein S15 [','30S ribosomal protein S17 [','30S ribosomal protein S19 [']
print(len(small_subunits))

#Notes: S12 needs fixing and regenerating

15


In [10]:
# In riboDB:
# ul1;ul2;ul3;ul4;ul5;ul6;ul10;ul13;ul14;ul15;ul18;ul22;ul23;ul24;ul29;
# us2;us3;us4;us5;us7;us8;us9;us10;us11;us12;us13;us14;us15;us17;us19;

In [11]:
!pwd

/nobackup1c/users/payette/ABGDtree5/sequences


In [19]:
#Manually upload zipped genomes to directory -- check
#Check the number of .gz genome files using the command line below.
#Checksum: should be ~387

!ls -ltr *.gz | wc

     43     387    4005


In [21]:
#Checksum: Should print 234 + 1 for GSulph addition + 79
# Checksum: 234-5 = 229 taxa in ABCDGtree4 list -2 of the plastid genomes = 227
#  More updates to ABGD taxa list

!ls -lR *.faa | wc -l

334


In [22]:
# Upload Excel file to create Pandas Dataframe of Taxa
taxa_df = pd.read_excel('ABGD_tree_taxa.xlsx') #Checksum: On 10-6-20 there should be 227
#7-6-21 plus 80 (-1 duplicate) = 306

In [23]:
#Re-index Pandas Dataframe to use the .faa filename for the index, used in loops
taxa_df.index=taxa_df.Filename 

In [24]:
#Print length of taxa dataframe and print last 5 records.
print(len(taxa_df))
taxa_df.tail()

306


,Index,Name,URL,Filename,FullURL,Accession,TaxID,Comments,TaxonomyNotes
Filename,,,,,,,,,
GCF_002268635.1_ASM226863v1_protein.faa,301,Cellvibrio mixtus,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_002268635.1_ASM226863v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,\tGCA_002268635.1 ASM226863v1,39650,NaN,NaN
GCF_001999945.1_ASM199994v1_protein.faa,302,Microbulbifer agarilyticus,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_001999945.1_ASM199994v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_001999945.1 ASM199994v1,260552,NaN,NaN
GCF_000179235.2_ASM17923v2_protein.faa,303,Synechococcus sp. CB0101,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000179235.2_ASM17923v2_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000179235.2 ASM17923v2,232348,NaN,NaN
GCF_014623465.1_ASM1462346v1_protein.faa,304,Marinomonas arctica,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_014623465.1_ASM1462346v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_014623465.1 ASM1462346v1,383750,NaN,NaN
GCF_001270065.2_ASM127006v2_protein.faa,305,Limnohabitans planktonicus II-D5,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_001270065.2_ASM127006v2_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_001270065.2 ASM127006v2,1293045,NaN,NaN


In [25]:
#For riboDB -- print out taxa list
for index,row in taxa_df.iterrows():
    print('#',row.Name.replace(' ', '_'))

# Simiduia_agarivorans_SA1_=_DSM_21679
# Congregibacter_litoralis_KT71
# Endozoicomonas_elysicola_DSM_22380
# Pseudomonas_aeruginosa_148
# Amphritea_japonica_ATCC_BAA-1530
# Halomonas_alkaliantarctica
# Alcanivorax_borkumensis_SK2
# Acinetobacter_baylyi_DSM_14961_=_CIP_107474
# Psychrobacter_aquaticus_CMS_56
# Alteromonas_macleodii_ATCC_27126
# Idiomarina_loihiensis_GSL_199
# Shewanella_oneidensis_MR-1
# Ferrimonas_balearica_DSM_9799
# Gallibacterium_anatis_UMN179
# Histophilus_somni_129PT
# Wigglesworthia_glossinidia_endosymbiont_of_Glossina_brevipalpis
# Wigglesworthia_glossinidia_endosymbiont_of_Glossina_morsitans
# Buchnera_aphidicola_str._APS_(Acyrthosiphon_pisum)
# Buchnera_aphidicola_str._Bp_(Baizongia_pistaciae)
# Escherichia_coli_str._K-12_substr._DH10B
# Yersinia_pestis_CO92
# Gilliamella_apicola
# Aeromonas_veronii_AER397
# Pseudoalteromonas_agarivorans_S816
# Rheinheimera_nanhaiensis_E407-8
# Francisella_noatunensis_subsp._noatunensis
# Coxiella_burnetii_RSA_493
# Legionell

In [26]:
#Code below will unzip automatically from a directory
#for index,row in taxa_df.iterrows():
#    subprocess.call(['gunzip', '%s/%s.gz' % ('/nobackup1c/users/payette/ABGDtree5/sequences',(index))])

In [27]:
#Should print 234. [240 - 6] - another 5 but those haven't been removed from the sequences yet
!ls -lR *.faa | wc -l

#306 = 334-26 extra files

334


In [28]:
!cd /nobackup1c/users/payette/ABGDtree5/sequences

In [29]:
#WARNING NAMES ARE CASE SENSITIVE!!!

In [30]:
taxa_df

,Index,Name,URL,Filename,FullURL,Accession,TaxID,Comments,TaxonomyNotes
Filename,,,,,,,,,
GCF_000305785.2_ASM30578v2_protein.faa,0,Simiduia agarivorans SA1 = DSM 21679,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000305785.2_ASM30578v2_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000305785.2,1117647,ABG Tree,"['tax_order=""Cellvibrionales""', 'tax_family=""C..."
GCF_000153125.2_ASM15312v2_protein.faa,1,Congregibacter litoralis KT71,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000153125.2_ASM15312v2_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000153125.2,314285,ABG Tree,"['tax_order=""Cellvibrionales""', 'tax_family=""H..."
GCF_000373945.1_ASM37394v1_protein.faa,2,Endozoicomonas elysicola DSM 22380,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000373945.1_ASM37394v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000373945.1,1121862,ABG Tree,"['tax_order=""Oceanospirillales""', 'tax_family=..."
GCF_000647595.2_PseAer148_2.0_protein.faa,3,Pseudomonas aeruginosa 148,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000647595.2_PseAer148_2.0_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000647595.2,1339263,ABG Tree,"['tax_order=""Pseudomonadales""', 'tax_family=""P..."
GCF_000381785.1_ASM38178v1_protein.faa,4,Amphritea japonica ATCC BAA-1530,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000381785.1_ASM38178v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000381785.1,1278309,ABG Tree,"['tax_order=""Oceanospirillales""', 'tax_family=..."
...,...,...,...,...,...,...,...,...,...
GCF_002268635.1_ASM226863v1_protein.faa,301,Cellvibrio mixtus,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_002268635.1_ASM226863v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,\tGCA_002268635.1 ASM226863v1,39650,NaN,NaN
GCF_001999945.1_ASM199994v1_protein.faa,302,Microbulbifer agarilyticus,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_001999945.1_ASM199994v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_001999945.1 ASM199994v1,260552,NaN,NaN
GCF_000179235.2_ASM17923v2_protein.faa,303,Synechococcus sp. CB0101,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000179235.2_ASM17923v2_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000179235.2 ASM17923v2,232348,NaN,NaN


In [24]:
#Taxa to fix

In [136]:
#taxa_df.loc['GCA_000018565.1_ASM1856v1_protein.faa']

In [139]:
#Sub routine which prints the number of completed genomes, based on index file of Pandas dataframe, for each protein.
protein_name='50S ribosomal protein L1 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df.iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            #print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

50S ribosomal protein L1 [
GCF_000305785.2_ASM30578v2_protein.faa
GCF_000153125.2_ASM15312v2_protein.faa
GCF_000373945.1_ASM37394v1_protein.faa
GCF_000647595.2_PseAer148_2.0_protein.faa
GCF_000381785.1_ASM38178v1_protein.faa
GCF_000712975.1_Version_1.2.07_protein.faa
GCF_000009365.1_ASM936v1_protein.faa
GCF_000046845.1_ASM4684v1_protein.faa
GCF_000471625.1_PsAq1.0_protein.faa
GCF_000172635.2_ASM17263v2_protein.faa
GCF_000401175.1_ASM40117v1_protein.faa
GCF_000146165.2_ASM14616v2_protein.faa
GCF_000148645.1_ASM14864v1_protein.faa
GCF_000209675.1_ASM20967v1_protein.faa
GCF_000011785.1_ASM1178v1_protein.faa
GCF_000008885.1_ASM888v1_protein.faa
GCF_000247565.1_ASM24756v1_protein.faa
GCF_000009605.1_ASM960v1_protein.faa
GCF_000007725.1_ASM772v1_protein.faa
GCF_000019425.1_ASM1942v1_protein.faa
GCF_000009065.1_ASM906v1_protein.faa
GCF_001693435.1_ASM169343v1_protein.faa
GCF_000297995.1_Aero_vero_AER397_V1_protein.faa
GCF_000363985.1_S816-1_1.0_protein.faa
GCF_000296695.1_ASM29669v1_protein.f

GCF_000258405.1_ASM25840v1_protein_EDIT_ADD_GENBANK_AFH48648.1.faa
GCF_000178855.1_ASM17885v1_protein_EDIT_ADD_GENBANK_EFH90870.1.faa
GCF_000279145.1_ASM27914v1_protein_EDIT_ADD_GENBANK_AFN73434.1.faa
GCF_003003775.1_ASM300377v1_protein_EDIT_PARTIAL.faa
GCF_000297255.1_ASM29725v1_protein_EDIT_PARTIAL.faa
GCF_000024985.1_ASM2498v1_protein_EDIT_RM_DUPE_WP_012870813.1.faa
GCA_000024525.1_ASM2452v1_protein_EDIT_NAMING.faa
GCA_000015705.1_ASM1570v1_protein_EDIT_NAMING.faa
GCF_000759855.1_ASM75985v1_protein.faa
GCF_000158595.1_ASM15859v1_protein.faa
GCF_000585215.1_Thiorhodococcus_sp._protein_EDIT_NAMING.faa
GCF_001469165.1_ML-1_protein_EDIT_RM_DUPE_WP_058557420.1_WP_082674593.1.faa
GCF_000280925.3_ASM28092v3_protein_EDIT_NAMING.faa
GCF_000024805.1_ASM2480v1_protein_EDIT_RM_DUPE.faa
GCF_000013405.1_ASM1340v1_protein_EDIT_ADD_UniProtKB_Swiss_Prot_Q2LQ85.2.faa
GCF_000012585.1_ASM1258v1_protein.faa
GCA_014377495.1_ASM1437749v1_protein_EDIT.faa
GCF_000648275.1_ASM64827v1_protein_EDIT_ADD_GENBANK

In [ ]:
len(taxa_df)

In [72]:
checkGenome('GCF_000170895.1_ASM17089v1_protein.faa')

50S ribosomal protein L1 [
>WP_045116851.1 50S ribosomal protein L1 [Plesiocystis pacifica]
 MGKKYKEARSKIDRDKRYGLVEACELVKVVAPAKFDETVECAARLGVNPRHADQMIRSSVSLPHGTGKSVRVLVFAKGDRVNQAKDAGADFVGSDDIVAKIQEGWLDFDVAIATPDMMGLVGRLGRILGPRNLMPNPKVGTVTMDVKKAVTERKAGRIEFRTERTGIIHAPIGKVSFEAVKLQENLETFLRTLQRLKPATAKGKYFKSVSLSSTMGPGIRLDTNEVQSMVTAE
1
50S ribosomal protein L2 [
>WP_006974417.1 50S ribosomal protein L2 [Plesiocystis pacifica]
 MGIKKYKPTSPGRRNMTSTDRAQVTAKKPFKKLVSINHRKAGRNNYGRITSRFRGGGHKRLYRQIDFKRDNYGVPGKVATIEYDPNRTTYIALIHYADGDKRYILAPEGLTVGDTIISSAKADIKPGNSMPLSAIPVGTIIHNIEIKIGRGGQFVRSAGVGARLMAREGNWTQIRLPSGEVRRVHVKCRATIGTLSNGEQANVNIGKAGRTRWLGRRPHQRGVVMNPVDHPMGGGEGRTSGGRNPTTPWGKPTKGYRTRHNKRTDVFIVSRRKK
1
50S ribosomal protein L3 [
>WP_198154501.1 50S ribosomal protein L3 [Plesiocystis pacifica]
 MGLLTKKIGMTEFYKEDGTRVPVTVLLAKGNVVTAHRTVERDGYTALQLAFDDQKASRLSKPKLGQFKKAEVSPKRKIKEFRVAADVVAQHAVGSEVDVSIFSDGQLVDVTGTSKGKGFQGVMKRHGMRGEKRTHGQHEVYRHGGSIGCRLTPGRVIPGKRMAGQMGNETVTVQNLEVAKVIADKGLILVKGAIPGGKNGYVQIRHAI

In [57]:
#Function which checks a genome for all proteins, prints 1 if complete
checkGenome('GCA_000018565.1_ASM1856v1_protein_EDIT.faa') 

50S ribosomal protein L1 [
>ABX02775.1 50S ribosomal protein L1 [Herpetosiphon aurantiacus DSM 785]
 MAKQHGKKYLAAAAKVEADRLYTPEEAAALVKETSFTNFDATVELHVRLGIDPRHADQTVRSTASLPHGTGKSVRVLVFAQGDTENIARNAGAEIVGGDELIQRIEKENFIDFDVAVATPDMMGKVGRLGRILGRRGLMPNPKSGTVVPPGDLPQAIKALQGGRVEFRNDKTGLLHVAIGKISFTPDQIRENLVVLLDAIKAAKPSGSKGVYIKSLTLTSTMGPGIPLDVNVAQATA
1
50S ribosomal protein L2 [
>ABX07548.1 50S ribosomal protein L2 [Herpetosiphon aurantiacus DSM 785]
 MGIKRYKPTSPGRRGMTVSDFAEITKFEPEKSLTEPLKKHAGRNNHGHITTRHRGGGHKRRYRLIDFKRKKFDIPAKVAGIEYDPNRSANIALLHYTDGEKRYIIAPLGLKVGDTLMAGPNAEIRVGNALPLANIPIGSQIHNIELTPGRGGQLVRSAGNTAQLMAREGNYATVRLPSGEMRMVHIKCMATLGQVGNVDHQNIMIGKAGRSRWLGRRPVVRGSAMNPVDHPHGGGEGRAPTGMNPKTKWGKPAMGKKTRHNPRTQRFIVRTRKQK
1
50S ribosomal protein L3 [
>ABX07545.1 50S ribosomal protein L3 [Herpetosiphon aurantiacus DSM 785]
 MLNGILGRKIGMTQIFTEKGETIPVTVIEAGPCVVTQLRTKDKDGYEAVQLGFGEIKPRKVTKPIQGHLKAAGRLVRFMREVKTTDLNAHNVGDVVNVDIFQIGEKIDVVGTSKGRGFAGVVKRHGFRGGPATHGQSDRHRAPGSIGSGTTPGRVWKNMRMAGRMGNDRVTVQNLEVVK

In [66]:
checkGenome('GCF_000020505.1_ASM2050v1_protein.faa')

50S ribosomal protein L1 [
>WP_012503111.1 50S ribosomal protein L1 [Chlorobaculum parvum]
 MAGKNYKNAAAKVDRAQEYELAEAIEKVKEITTTKFDATIDVAMKLGVDPRHADQVVRGTVMLPHGTGKTVSVLVVCKENKAEEAREAGADFVGFEDYIEKIQNGWTDVDVVVATPDVMGQLGKVARILGPRGLMPNPKSGTVTMDVAKAVSEVKAGKIEFRVDKAGNIHAPVGKVSFDSDKLAGNIGSFIKEVVRLKPSAAKGQYLQGITISSTMSPGVKVKKDKFVA
1
50S ribosomal protein L2 [
>WP_012501442.1 50S ribosomal protein L2 [Chlorobaculum parvum]
 MAVRKLAPNTPGTRFASYASFEEITKSEPEKSLLVPLKKSGGRNTNGRITSRHMGGGHKRFYRLIDFKRNKDNVPAKVAAIEYDPNRSARIALLHYVDGEKRYILAPKGLKVGDRIESGEKVDIRVGNTMPLKNIPLGTDVHNIEMKAGKGGQMARSAGSYAVLAAREGVHATLKLPSGEIRKVRVECRATIGVIGNSEHENISLGKAGRSRWLGIRPQTRGMAMNPVDHPMGGGEGKSKSGGGRKHPKSPWGQLAKGQKTRNKKKASTKLIVRGRKAK
1
50S ribosomal protein L3 [
>WP_012501439.1 50S ribosomal protein L3 [Chlorobaculum parvum]
 MGAILGKKIGMTRLYNNKREAVPCTVIQAGPCFVAQVKSTEKDGYDAYQLGFGERDEKKVTKPLAGHYKKAGKTPGYLLSEVSKSLIAGELEAGATVAVDVFTEGEIVDVLGVTKGKGFAGVVKRHNFGGGSRTHGQSDRLRAPGSVGGSSDPSRTFKGTRMAGRMGGENKTVRNLEIVKVMPESNLIVVKGAVPGPKNSYVKIVSTTK


In [25]:
### Checking Routine below

In [29]:
protein_name='50S ribosomal protein L5 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

50S ribosomal protein L5 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_010886968.1 50S ribosomal protein L5 [Deinococcus radiodurans]
 MQQLKTKYNDQVRPALMQQFGYSSVMAVPRIEKIVVNEGLGSSKEDSKAIDKAAKELALITLQKPIITKAKKSISNFKLRQGMPVGIKVTLRGERMYVFLEKLINIGLPRIRDFRGINPNAFDGRGNYNLGIKEQLIFPEITYDMVDKTRGMDITIVTTAKTDEEARALLQSMGLPFRKQ
GCF_000190555.1_ASM19055v1_protein.faa
>WP_013614185.1 50S ribosomal protein L5 [Deinococcus proteolyticus]
 MLQLKEKYQSEVRPALVEQFSYSSVMAAPRIEKIVINEGLGSAARDDSKAIDKAAKELALITLQKPVITKAKKSISNFKLRQGMPIGVKVTLRGDRMWVFLDKLINVGLPRIRDFRGINPNAFDGRGNYNLGIKEQLIFPEITYDMVDRTRGMDITIVTTAKTDEEARALLQGLGLPFRR
GCF_000091545.1_ASM9154v1_protein.faa
>WP_011228843.1 MULTISPECIES: 50S ribosomal protein L5 [Thermus]
 MPLDVALKRKYYEEVRPELIRRFGYQNVWEVPRLEKVVINQGLGEAKEDARILEKAAQELALITGQKPAVTRAKKSISNFKLRKGMPIGLRVTLRRDRMWIFLEKLLNVALPRIRDFRGLNPNSFDGRGNYNLGLREQLIFPEITYDMVDALRGMDIAVVTTAETDEEARALLELLGFPFRK
GCF_000376665.1_ASM37666v1_protein.faa
>WP_013014855.1 MULTISPECIES: 50S ribosomal protein L5 [Meiothermus]

GCF_001748285.1_ASM174828v1_protein.faa
>WP_069805728.1 50S ribosomal protein L5 [Thermogemmatispora onikobensis]
 MAARLKERYLKEVVPALQKEFNYRNPMQVPRIHKVVINIGMGEAIQNAKAMEAAVTDLSLITGQRPVITHARRSVASFKVRKGMQIGCMVTLRGERMYYFLDKLMNVALPRLRDFQGVSPDSFDGRGNYTLGLREQLVFPEIDYDKIDKVRGMEVSIVTTARTDEEGRRLLSLLGMPFRR
GCF_000170895.1_ASM17089v1_protein.faa
>WP_006974408.1 50S ribosomal protein L5 [Plesiocystis pacifica]
 MAKAAATQKKKPEAGYAFKFPKVRNEPVDRGDFVPHTRTLYREKVIPALKEEFGYKNPNTIPTVEKVVLNIGLGEAIKQPKLLEQSFECLGNITGQRPVVTRAKKSIAGFKLREGMKIGAMVTLRRTAMWDFLDRLLMVALPRTRDFRGVSRKAFDGRGNYTLGIKEILIFPEVDIEKLNAVPGMNVCIHTSAKTDDEARALLKHLGMPFRQ
GCF_001273775.1_ASM127377v1_protein.faa
>WP_053382003.1 50S ribosomal protein L5 [Nitrospira moscoviensis]
 MYQQQVVPALMKEFGYKNVMQVPKLERVVLNVGMGEAIQNVKLLESAVAELGTITGQKPVVTRAKKAIAGFKLRQGLPIGAKVTLRSRRMYEFFDRLVTLALPRIRDFRGVSPKAFDGRGNYTLGLKEQLIFPEIKYDEVASIHGMDITIVTTARTNDEGKALLKHLGMPFRT
GCF_000025705.1_ASM2570v1_protein.faa
>WP_013028916.1 50S ribosomal protein L5 [Sideroxydans lithotrophi

>WP_013695852.1 50S ribosomal protein L5 [Candidatus Pelagibacter sp. IMCC9063]
 MIPRLKKIYLEEILPNIVKNLSIKNTMQAPKIEKIVLNMGLGLDASDSKILKIIKEDLANLTGQLPIVTKSKKAVANFKTRKDIPLGLKVTLRKNNMYYFLDRLINMALPRIKDFRGLNPDSFDSEANYSFGIKEHVIFPEVSFDKIEKLRGLDITVVTSTNNKEASKALLDSFDFPFYKKGSN
GCF_003350535.1_ASM335053v1_protein.faa
>WP_114769219.1 MULTISPECIES: 50S ribosomal protein L5 [Microvirga]
 MAKAQLDGYTPRLKKHYEEVVRPKLIEEFGYKNPMEVPAIDKIVLNMGVGESTADSKKASVAAADLALIAGQKPVITKARKAISQFKVREGMPIGAKVTLRKARMYEFMDRLVTVALPRVRDFRGLNPKSFDGRGNYALGIKEHLVFPEINYDKAEQVWGMDVIIATTAKTDEEARALLRHFNFPFRQ
GCF_000702665.1_ASM70266v1_protein.faa
>WP_029459052.1 50S ribosomal protein L5 [Desulfovibrio alcoholivorans]
 MTRLEKIYAEKVAPELKKAFGYTSSMQIPRLSFISLNMGLGEASNNNKLIEEAVVELTAISGQKAVVTRARKSIAAFKLREGMPVGCRVTLRRDRMWDFLDKLINFALPRVRDFRGVPDRGFDGRGNFTLGVREHTIFPEINVDRVEHVKGMNVTIVTTACADKEGKMFLDLLGMPFKK
GCF_000010665.1_ASM1066v1_protein.faa
>WP_006920479.1 MULTISPECIES: 50S ribosomal protein L5 [Desulfovibrio]
 MTRLEQIYAEKVAPELKKEFGYTSSMQIPRLSFV

In [41]:
protein_name='50S ribosomal protein L24 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

50S ribosomal protein L24 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_187767769.1 50S ribosomal protein L24 [Deinococcus radiodurans]
 MHFKKGDTVIVLSGKHKGQTGKVLLALPRDQKVVVEGVNVITKNVKPSMTNPQGGQEQRELALHASKVALVDPETGKATRVRKQIVDGKKVRVAVASGKTID
GCF_000190555.1_ASM19055v1_protein.faa
>WP_174260423.1 50S ribosomal protein L24 [Deinococcus proteolyticus]
 MHVRKGDTVVVLSGKHKGATGKVLETSPRDHKVLVEGVNTVIKHVKPSFANPEGGRVEKEIALHASKVALVDPETGKATRVRKQIVDGRKVRVAAASGKVID
GCF_000091545.1_ASM9154v1_protein.faa
>WP_011173707.1 50S ribosomal protein L24 [Thermus thermophilus]
 MRVKMHVKKGDTVLVASGKYKGRVGKVKEVLPKKYAVIVEGVNIVKKAVRVSPKYPQGGFIEKEAPLHASKVRPICPACGKPTRVRKKFLENGKKIRVCAKCGGALDTEE
GCF_000376665.1_ASM37666v1_protein.faa
>WP_013014856.1 MULTISPECIES: 50S ribosomal protein L24 [Meiothermus]
 MKVHVKKGDTVIVLSGKEGLRGETGKVKAVMPKEGLVVVEGLNLIKRAVRPNPRNPQGGFIETEAPIHASKVMPICPSCEKPTRIRKKVLPDGTKVRACAKCDGVLDTK
GCF_000195955.2_ASM19595v2_protein.faa
>NP_215229.1 50S ribosomal protein L24 [Mycobacterium tuberculosis H37Rv]

GCF_001273775.1_ASM127377v1_protein.faa
>WP_053378952.1 50S ribosomal protein L24 [Nitrospira moscoviensis]
 MGDVVQALGKSRIRKGDTVVVIAGRERGKSGKVLSVDLGAGKVTVEKLNIIKRHTKPNQKAKQGGILEREAPLAISNVMFLCPVTQKPTRMGIRTLADGRRVRFSKKSNETIE
GCF_000025705.1_ASM2570v1_protein.faa
>WP_013028915.1 50S ribosomal protein L24 [Sideroxydans lithotrophicus]
 MNKIKKNDDVIVIAGKDKGNRGSVLNVLGNDRVLVSGVNKVKKHQKPNPMKGTTGGIVEIEKPIHVSNIAIYNSATKKADRVGVKQLEDGRKVRVFKSSGEVIA
GCF_004365595.1_ASM436559v1_protein.faa
>WP_066805538.1 50S ribosomal protein L24 [Pseudodesulfovibrio indicus]
 MMKTKIRKDDKVMVIAGKDKGKVGKVLKILKKQDKVLVEKVNMVQRHTKANPYAQQPGGIIEKEAPIHVSNVAVVCDACTKPTRVGYKKTEDGKKVRFCKKCNETFK
GCF_000266945.1_ASM26694v1_protein.faa
>WP_014812962.1 50S ribosomal protein L24 [Desulfomonile tiedjei]
 MGNIKKNDKVKVISGKEKGRQGKILKVLGEKDAALVERLNLVKRHTKAGRAGQQGGIVEKEAPIKLSKLMLVCPKCSKPTRTGSRVLDDGKRVRYCKKCSEQLES
GCF_000014965.1_ASM1496v1_protein.faa
>WP_011698423.1 50S ribosomal protein L24 [Syntrophobacter fumaroxidans]
 MKGQKSYRIKKNDTVM

In [58]:
protein_name='30S ribosomal protein S4 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

30S ribosomal protein S4 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_010888758.1 30S ribosomal protein S4 [Deinococcus radiodurans]
 MGRFRGSITKQSRREGINLAETEKVQKYLDRRPYGPGQHGQRRKGRPSDYSVRLREKQKLARLYGMNEKQFRNLFEEASNVPGVTGTVFLQLLERRLDNVVFRMGFASTRRQARQFVGHGHILVNGKKVDIASYRVKIGDEISVAEGSRSMGFIQENMDAQKRRRVSPWIEMDQDNFKGTFSRLPAREDLALPINENFIIEYYSR
GCF_000190555.1_ASM19055v1_protein.faa
>WP_013614200.1 30S ribosomal protein S4 [Deinococcus proteolyticus]
 MGRFSGSIVKQSRREGINLAETEKVQKYLDRRPYAPGQHGQRRGRGRPSDYSVRLREKQKLSRLYGMNEKQFRNLFEEATRQPGVLGTVFLQLLERRLDNVVFRMGFASTRRQARQFVGHGHILVNGKKVDIASYRVKIGDEITVAEKSRSMGFVQENMEAMKRRRPSPWLEVNAETFTGTFNRLPAREDLALPINENFIIEYYSR
GCF_000091545.1_ASM9154v1_protein.faa
>WP_011173699.1 30S ribosomal protein S4 [Thermus thermophilus]
 MGRYIGPVCRLCRREGVKLYLKGERCYSPKCAMERRPYPPGQHGQKRARRPSDYAVRLREKQKLRRIYGISERQFRNLFEEASKKKGVTGSVFLGLLESRLDNVVYRLGFAVSRRQARQLVRHGHITVNGRRVDLPSYRVRPGDEIAVAEKSRNLELIRQNLEAMKGRKVGPWLSLDVEGMKGKFLRLPDREDLALPVNEQLVIEFYSR
GCF_000376665.1_ASM37666v1_pro

>WP_045116839.1 30S ribosomal protein S4 [Plesiocystis pacifica]
 MARYTGPVCRLCRREDQKLFLKGDRCFSEKCAYDRRQYPPGQHGQGRRRRPSDYGHQLREKQKVKRMYGLLEKQFRGYYFKAARMKGITGENLLSLLERRLDNVAVRCGFAASHAEARQLVRHGHFTINGKKVNIPSYQVREGEEIVVREKSRKIQKIVDALAKLDRSPLPTWLEIDRDNFKAKIKNVPQRSDISADIDEQLIVELYSK
GCF_001273775.1_ASM127377v1_protein.faa
>WP_053378965.1 30S ribosomal protein S4 [Nitrospira moscoviensis]
 MAKYRGPVCRLCRREGEKLFLKGSRCMTEKCAIERRSYPPGQHGQGRPRTSDYSLQLREKQKLRRIYGLQERQFRSVFERAERQSGVTGDALLRLLECRLDNVAYRLGFAASRKQARQIVSHGHLTLNGKKITVAGALVKAGDVIEVRERSRNLVAIQSALEAVDGRGVPEWLELDKGAFKGTVRALPAKEHIALPVNEQMVVELYSR
GCF_000025705.1_ASM2570v1_protein.faa
>WP_013028929.1 30S ribosomal protein S4 [Sideroxydans lithotrophicus]
 MARNLDAKCRQCRREGEKLFLKAEKCFTDKCSVERRAYAPGQHGQKKNQRQSEYGGQLREKQKLRRIYGMMEGQFRLTYKRAEQSRGITGESLLQMLESRLDNVSYRMGFGASRSEARQVVRHNSILVNGKRVNIPSYQVRPGDVVEVAEKSRNQLRVKAALQAAEQRGFPEWLEVDAKAFKGTFKAVPQRSELPATINEHLVVELYSK
GCF_004365595.1_ASM436559v1_protein.faa
>WP_066805511.1 30S ribosomal protein S4 [Pseud

>WP_013695903.1 30S ribosomal protein S4 [Candidatus Pelagibacter sp. IMCC9063]
 MTKRIQAKHKVDRRLGVNLWGRPKSPFNTRAFGPGQHGQTRARGKPSDYKIQLQAKQKLKGYYGNLNERQFRNVFQEAERVRGNTGENLVGILESRLDAVVYRAKFALTVFQARQIINHGHIKVNGKKVNVGSYSLKENDEIEVRDSSKQLTTILSSVASKEREVPEYVVVDHKQLTAKYTRVPKMEEIPYPAIMEPNLVVEYYSR
GCF_003350535.1_ASM335053v1_protein.faa
>WP_114769108.1 MULTISPECIES: 30S ribosomal protein S4 [Microvirga]
 MSKRIQAKHKLDRRMGQNIWGRPKSPVNRREYGPGQHGQRRKGKLSDFGTQLRAKQKLKGYYANITEKQFRRYYAEAIRLKGDSGENLVGLLERRLDAVVYRAKFVSTPFAARQFVNHGHVKVNGRRVNIPSYLVKPGDVIEVKEKSKQLEVVVVASQLAERDVPDYFEVDHSKMTARMTRVPTLSEVPYPVQMEPNLVIEFYSR
GCF_000702665.1_ASM70266v1_protein.faa
>WP_029459064.1 30S ribosomal protein S4 [Desulfovibrio alcoholivorans]
 MARYTEAKCRICRREGAKLFLKGDRCYTDKCAYERRPYAPGQHGRIRKKMSDYAIQLREKQKTRRMYGILEGQFRAYFHRADMKTGVTGENLLSFLERRMDNVIYRLGFANSRNQARQLVRHGIFTLNGHRVTIPSMQVKVGDVIEVRERNRQSPIILEAQQVIARRGCPAWLEVDGEKLKGKVNALPTREDVQFPINEQLIVELYSK
GCF_000010665.1_ASM1066v1_protein.faa
>WP_006920491.1 MULTISPECIES: 30S ribo

In [60]:
protein_name='30S ribosomal protein S11 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

30S ribosomal protein S11 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_010888757.1 MULTISPECIES: 30S ribosomal protein S11 [Deinococcus]
 MAKPTKGKAPRRSRRNISAGRAYVHASYNNTIVTITDLDGNSVAWSSGGTIGYKGSKKGTPYAAQLAAADAVKKAQTSFGMAAVDVIVRGSGSGREQAIRAIQASGIEVRSIMDDSPVPHNGCRPKKKFRA
GCF_000190555.1_ASM19055v1_protein.faa
>WP_013614199.1 30S ribosomal protein S11 [Deinococcus proteolyticus]
 MAKTTKGRAPRRSRRSIAAGRAYVHASYNNTIVTITDQDGNSVAWSSGGTIGYKGSKKGTPYAAQLAAADAVKKAQQSFGMNSVDVIVRGSGSGREQAIRAIQASGIEVRSIMDDTPVPHNGCRPKKKFRA
GCF_000091545.1_ASM9154v1_protein.faa
>WP_008633365.1 MULTISPECIES: 30S ribosomal protein S11 [Thermus]
 MAKKPSKKKVKRQVASGRAYIHASYNNTIVTITDPDGNPITWSSGGVIGYKGSRKGTPYAAQLAALDAAKKAMAYGMQSVDVIVRGTGAGREQAIRALQASGLQVKSIVDDTPVPHNGCRPKKKFRKAS
GCF_000376665.1_ASM37666v1_protein.faa
>WP_013014841.1 MULTISPECIES: 30S ribosomal protein S11 [Meiothermus]
 MAEKKSATSRKKKIKRQVTTGKAFIHASYNNTIVTITDNDGHPVTWSSGGVIGYKGSRKGTPYAAQLAAMDAAKKAQGYGMSSVEVVVRGTGAGREQAIRALQASGLQVRSIVDDTPTPHNGCRPRKKFRKAV
GCF_0001

GCF_001273775.1_ASM127377v1_protein.faa
>WP_053378964.1 30S ribosomal protein S11 [Nitrospira moscoviensis]
 MSVKKGKKKERRIVQSGVAHVQASFNNTIVTITDMSGNTVVWASAGNQGFKGSRKSTPFAAQRAGEAAARKAMESGMRQIDVYVNGPGSGRESAIRSLQAAGLRINLIRDVTPIPHNGCRPPKRRRV
GCF_000025705.1_ASM2570v1_protein.faa
>WP_013028928.1 30S ribosomal protein S11 [Sideroxydans lithotrophicus]
 MAKPSTRVRKKVKKNVAEGIAHVHASFNNTIVTITDRQGNTLAWSTSGAQGFKGSRKSTPFAAQIAAETVGKSAVECGVKNLEVRIKGPGPGRESAVRALNAAGFKITSISDITPVPHNGCRPPKKRRI
GCF_004365595.1_ASM436559v1_protein.faa
>WP_066805513.1 30S ribosomal protein S11 [Pseudodesulfovibrio indicus]
 MAKPRRSGKKKEKKNIPVGIAHVKATFNNTIVTFTDVKGNVVSWASAGAHFKGSRKSTPFAAQMAAEAAAKRAQDSGMRTVGIYVKGPGSGREAAMRAINNAGFKVTFIRDITPIPHNGCRPPKRRRV
GCF_000266945.1_ASM26694v1_protein.faa
>WP_014812975.1 30S ribosomal protein S11 [Desulfomonile tiedjei]
 MPRRRVVRKKKEKRIIPRGVAHIQSTFNNTIVTITDPEGNVVAWSSSGTQGFKGSRKSTPFAAQLAAEDAARKAMEQGMRIVEVLIKGPGAGREAALRSLQAAGFQISLIKDVTPIPHNGCRPPKRRRV
GCF_000014965.1_ASM1496v1_protein.faa
>WP_01

In [63]:
protein_name='30S ribosomal protein S14 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

30S ribosomal protein S14 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_027479943.1 30S ribosomal protein S14 [Deinococcus radiodurans]
 MANKSKLAKQKQREKTVEKYAAKRAELKAAGDYYGLTQLPRDASPTRLHNRCEFTGRPRGYVRFFGVSRIVLREMAHRGELPGVKKASW
GCF_000190555.1_ASM19055v1_protein.faa
>WP_013614186.1 30S ribosomal protein S14 [Deinococcus proteolyticus]
 MAKKSKIAKQKQREQKVAKYAAKRAEMKAAGDYYGLSQLPRDASPTRLHNRCEFTGRPRGYIRHFGVSRIVMREMAHRGELPGVRKASW
GCF_000091545.1_ASM9154v1_protein.faa
>WP_008633399.1 MULTISPECIES: type Z 30S ribosomal protein S14 [Thermus]
 MARKALIEKAKRTPKFKVRAYTRCVRCGRARSVYRFFGLCRICLRELAHKGQLPGVRKASW
GCF_000376665.1_ASM37666v1_protein.faa
>WP_013014854.1 MULTISPECIES: 30S ribosomal protein S14 [Meiothermus]
 MAKKSQVEKMKRKLKTIAKYAAKRAALKAAGDYAALAELPRDASPTRHKNRCAVTGRSKAYMRYFGLSRLQFREMAHKGQLPGVKKASW
GCF_000195955.2_ASM19595v2_protein.faa
>NP_216572.1 30S ribosomal protein S14 [Mycobacterium tuberculosis H37Rv]
 MAKKSKIVKNQRRAATVARYASRRTALKDIIRSPSSAPEQRSTAQRALARQPRDASPVRLRNRDAIDGRPRGHLRKFGLSRVRVR

>WP_015867798.1 MULTISPECIES: 30S ribosomal protein S14 [Variovorax]
 MAKQSLIQRELKRDKLVAKYAAKHAELKAISNDLKKSDEERAAARLGLQKLPRNANPTRQRNRCEITGRPRGTFRQFGLARAKIRELAFNGDIPGVTKASW
GCF_000473895.1_KS51_v1_protein.faa
>WP_022606567.1 30S ribosomal protein S14 [Rubidibacter lacunae]
 MAKKSMIARERKREKLVAKYSAKREELKEQIRTAQTPLERFELRRKLQNLPRNSSPTRLHNRCKVTGRPRGYYRDFGLSRNVLREWAHEGLLPGVTKSSW
GCF_000317555.1_ASM31755v1_protein.faa
>WP_015189741.1 MULTISPECIES: 30S ribosomal protein S14 [Chroococcaceae]
 MAKKSMIEREKKRQKLVTKYAQKREALIEQFEQATSQREKLEIHRQIQQLPRNSAPTRLHNRCWVTGRPRGVYRDFGLSRHVLREWAHQGLLPGVVKSSW
GCF_001189295.1_ASM118929v1_protein.faa
>WP_082362215.1 type Z 30S ribosomal protein S14 [Chondromyces crocatus]
 MARAKEFAKLNRPPKFAVRHRNRCKVCGRSRAYYRDFELCRVCLRLFALRGELPGVIKASW
GCF_000067165.1_ASM6716v1_protein.faa
>WP_012240559.1 type Z 30S ribosomal protein S14 [Sorangium cellulosum]
 MARAKEFAKLNRPPKFSTRHKNRCKVCGRARAYYRDFELCRVCLRLFALRGELPGVIKASW
GCF_002142475.1_ASM214247v1_protein.faa
>WP_026148789.1 MULTIS

In [65]:
protein_name='50S ribosomal protein L22 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

50S ribosomal protein L22 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_010886961.1 50S ribosomal protein L22 [Deinococcus radiodurans]
 MTAPEQTFRNKKQRKQQVKLRKPGFAVAKYVRMSPRKVRLVVDVIRGKSVQDAEDLLRFIPRSASEPVAKVLNSAKANALHNDEMLEDRLFVKEAYVDAGPTLKRLIPRARGSANIIKKRTSHITIIVAEKGNK
GCF_000190555.1_ASM19055v1_protein.faa
>WP_013614178.1 50S ribosomal protein L22 [Deinococcus proteolyticus]
 MTATQTAEFRNKKQRKQEHKLRKPGFARAKYVRMSPRKVRLVVDLIRGKSVRDAEDILRFLPHIASEPISKVLNSAKANALHNDDMLEDQLFIKEAYVDAGPTLKRLIPRARGAANIIRKRTSHITIVVGEKGAN
GCF_000091545.1_ASM9154v1_protein.faa
>WP_011173710.1 MULTISPECIES: 50S ribosomal protein L22 [Thermus]
 MEAKAIARYVRISPRKVRLVVDLIRGKSLEEARNILRYTNKRGAYFVAKVLESAAANAVNNHDMLEDRLYVKAAYVDEGPALKRVLPRARGRADIIKKRTSHITVILGEKHGK
GCF_000376665.1_ASM37666v1_protein.faa
>WP_041654122.1 50S ribosomal protein L22 [Meiothermus ruber]
 MATARYVRMSPRKVRLVVDLIRGKKLEEARAILKYTPHRASEVVAKVLESAAANGMNDDRKNMIEDQIYVKAAYVDEGPAIKRVLPRARGSANMIKKRTSHITIIVGEKNG
GCF_000195955.2_ASM19595v2_protein.faa
>NP_215220

GCF_001273775.1_ASM127377v1_protein.faa
>WP_053378947.1 50S ribosomal protein L22 [Nitrospira moscoviensis]
 MAEAKAILRFVRVAPRKAKPVIDMIRGQQVPMALAMLKHTPRHAARVVEKILKSAVANAEQKEMGDSESMVISKAFVDCGPTYKRFRARSMGRANAIQKRTSHITVVVAAPSLDGKRK
GCF_000025705.1_ASM2570v1_protein.faa
>WP_190272146.1 50S ribosomal protein L22 [Sideroxydans lithotrophicus]
 MMSTTAVIRGVRLSAQKARLVADQIRGLPVAQALNILAFSPKKGGGIIKKALDSAIANAEHNDGADIDELKVATIYVDKGTSLKRMIPRAKGRGNSIEKQTCHITITVGS
GCF_004365595.1_ASM436559v1_protein.faa
>WP_066805560.1 50S ribosomal protein L22 [Pseudodesulfovibrio indicus]
 MEAKAVAKFIRVSPRKTRVVAENIKGKGVEDALNILRFTPKKPAEILRKVLYSAISNAEQMPGVDVDSLIVDTVMVNEGPTWKRIKPRAMGRAYRIRKRTSHITIVVKEQ
GCF_000266945.1_ASM26694v1_protein.faa
>WP_014812956.1 50S ribosomal protein L22 [Desulfomonile tiedjei]
 MEWVSTHKYARISARKARLVADLVRGMPVSKAMSLLRFTPKKGAELILKVVQSALGNATQSRGVDEESLYISRIMIDEGPTAKRWIPRAMGRATRVRKRTSHIVVAVSEE
GCF_000014965.1_ASM1496v1_protein.faa
>WP_011698417.1 50S ribosomal protein L22 [Syntrophobacter fumaroxidans

In [67]:
protein_name='50S ribosomal protein L23 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

50S ribosomal protein L23 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_010886958.1 50S ribosomal protein L23 [Deinococcus radiodurans]
 MSHYDILQAPVISEKAYSAMERGVYSFWVSPKATKTEIKDAIQQAFGVRVIGISTMNVPGKRKRVGRFIGQRNDRKKAIVRLAEGQSIEALAGQA
GCF_000190555.1_ASM19055v1_protein.faa
>WP_013614175.1 50S ribosomal protein L23 [Deinococcus proteolyticus]
 MSHYDIIQAPVVSEKAFAGMEKGVYSFWVSPKATKPEIRDAIQKAFGVQVTGISTMNVRGKRKRVGRFMGQRNDRKKAVVRLADGQKIEALEGLV
GCF_000091545.1_ASM9154v1_protein.faa
>WP_008633421.1 MULTISPECIES: 50S ribosomal protein L23 [Thermus]
 MKTAYDVILAPVLSEKAYAGFAEGKYTFWVHPKATKTEIKNAVETAFKVKVVKVNTLHVRGKKKRLGRYLGKRPDRKKAIVQVAPGQKIEALEGLI
GCF_000376665.1_ASM37666v1_protein.faa
>WP_013014865.1 MULTISPECIES: 50S ribosomal protein L23 [Meiothermus]
 MKTPHDVIIQPVLSEKAYSRFADGVYTFWVHPKANKTEIANAVEAAFKVKVVRVNTQNVLGKHKRLGRFMGKRPDRKKAIVTVAPGQKIEALEGLI
GCF_000195955.2_ASM19595v2_protein.faa
>NP_215217.1 50S ribosomal protein L23 [Mycobacterium tuberculosis H37Rv]
 MATLADPRDIILAPVISEKSYGLLDDNVYTFLVRPDSN

GCF_000473895.1_KS51_v1_protein.faa
>WP_022605551.1 50S ribosomal protein L23 [Rubidibacter lacunae]
 MVPDYNPRDLIDLVHRPIVTEKATLLLEENKYVFDVTKRATKPQVKAAIELLFDVKVTKVNLLRQPPKRRRVGRFIGYKSQYKRAIVTLAEGDTIVLFPDV
GCF_000317555.1_ASM31755v1_protein.faa
>WP_015187440.1 MULTISPECIES: 50S ribosomal protein L23 [Chroococcaceae]
 MPKIDRRSLPDLVRRPILTEKATRLMEENKFTFEVTPKATKPQIKAAIEDLFEVKVVQVNTQQQPRKQRRVGKFIGFKPQYKKAIVTVATGEEDKIRKVLFPDV
GCF_001189295.1_ASM118929v1_protein.faa
>WP_050430054.1 50S ribosomal protein L23 [Chondromyces crocatus]
 MQLTPEQIIRRPIILTEKSARLREGDNKVIFEVRREANKIQIKDAIQTLFKVGVLDVNTMVLRGKDKRMGRGYAKLRNWKKAIVTLKAGDEIQFFDEEKAE
GCF_000067165.1_ASM6716v1_protein.faa
>WP_012233478.1 50S ribosomal protein L23 [Sorangium cellulosum]
 MQPEQIIRRPIILTEKSSRLRDQGNKVIFEVRREANKIQIKDAIQTLFKVGVVDVNTLIMRGKDKRMGRGYAKLRNWKKAIVTLKEGDEIQFFDEKAE
GCF_002142475.1_ASM214247v1_protein.faa
>WP_017286259.1 MULTISPECIES: 50S ribosomal protein L23 [Leptolyngbya]
 MKTDPRNLADLIRRPLITEKATRLLELNQYTFEVMPKASKPEIRAAIEYLFDVK

In [68]:
protein_name='50S ribosomal protein L10 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

50S ribosomal protein L10 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_010888676.1 50S ribosomal protein L10 [Deinococcus radiodurans]
 MANEKNQQTLGSLKDSLQGIETFYVVDYQGLTAGQLTQLRKDIREKGGQLIVAKNTLLNLALQEGGRDFDDALKGPSALVLAQEDPAGVAKALSDAAGRNDRGIPTVKGGFVEGSKVDVAVVQRLASLGSKTTLQAELVGVLSAHLSNFVGILEAYREKLEGEGGSESA
GCF_000190555.1_ASM19055v1_protein.faa
>WP_013614644.1 50S ribosomal protein L10 [Deinococcus proteolyticus]
 MANERNQKSLSDLQQSLAGVESFYLVDYQGLSAGQLSKLRQDIRDKGGQLIVAKNTLIHLALQSNDQDFSDILAGPSALVLASEDPAGVAKALSDIAKTNEAGIPTFKGGFVEGNRVDTKVITKLATLGSKQSLQAEMVGVLSTHLATFIGTLEAYQSKLQAEGGSAPAPAAE
GCF_000091545.1_ASM9154v1_protein.faa
>WP_008630583.1 MULTISPECIES: 50S ribosomal protein L10 [Thermus]
 MPNKRNVELLATLKENLERAQGSFFLVNYQGLPAKETHALRQALKQNGARLFVAKNTLIRLALKELGLPELDGLQGPSAVVFYEDPVAAAKTLVQFAKSNPKGIPQVKSGLLQGQILTAKDVEALAELPTMDELRAELVGVLQAPMAELVGVLGGVARELVGILEAYAEKKAA
GCF_000376665.1_ASM37666v1_protein.faa
>WP_013013456.1 50S ribosomal protein L10 [Meiothermus ruber]
 MPSKRNIENLEALTAALKGAGGSFFVVN

>WP_006974428.1 50S ribosomal protein L10 [Plesiocystis pacifica]
 MEKSIKYAQSAALREEVEDLTALVVVEYDRLTVAEAEELRGQFRDAGCTYRVYKNSVIRYAIAGTSHEPLSPFLKGVSGLAYNADDPGAPARVARDFAKKNDKLRLKAGVVEGEVLEGEACNKLADMPGPREIKAKLLMLFKTPATRMVQVMQAAPRDFLRVLNAKKKKDEEAA
GCF_001273775.1_ASM127377v1_protein.faa
GCF_000025705.1_ASM2570v1_protein.faa
>WP_013028896.1 50S ribosomal protein L10 [Sideroxydans lithotrophicus]
 MALNLQEKQAVVAEVSAQVAQAQTIVVAEYRGIPVADITSLRANARKNGVYLHVLKNTLARRAVQGTSFEGLADKMVGPLMYSMSADAVAAAKVVYEFAKTNDKLVIKGGAYNGKVLDAAAVSALASVPSKEVLLAQLLGVMKSPITSMAIVLKALSEKKAGAAA
GCF_004365595.1_ASM436559v1_protein.faa
>WP_066805585.1 50S ribosomal protein L10 [Pseudodesulfovibrio indicus]
 MNRQEKAQIIEQLHEKASRASIAVVTDFKGLTVEEMTVLRSKCFEIGVDYQVVKNTLARLALNDTSHGELSEHLKENCAIALGYEDPVALAKALADYAKTNKKFALRYGTLEGQFLDSDAVKELAKMPSKPELLSSLLGTMQAVPRNFVCLFANIERKFLYALTAIKEQKEAA
GCF_000266945.1_ASM26694v1_protein.faa
>WP_014812942.1 50S ribosomal protein L10 [Desulfomonile tiedjei]
 MKREEKEKQVAWLREELQEAKALFLTNYQGLSVSEMNSLRSELRSKGI

GCF_000346485.2_ASM34648v2_protein.faa
>WP_017740589.1 50S ribosomal protein L10 [Scytonema hofmannii]
 MPRTIEDKHAIVADLKETLSQSQLAIVIDYQGLTVAEITDLRRRLRPAGTVCKVTKNTLMGIAIKDQEKWQPLEELLKGSSAFLLVKEDFSAAIKAYQDFQKVTKKTEVRGGVMEGRLLKEPDIKALGDLPSKEQLMAQIAGALNALTTKIAVGINEVPSSLARALQAVADKEKGDGETESASS
GCF_001617355.1_Rhodanobacter_sp._FW104-T7_protein.faa
>WP_007514535.1 MULTISPECIES: 50S ribosomal protein L10 [Rhodanobacter]
 MALNLSQKQEVVAELAEVAAKAHSLVAAEYAGTTVAQMTAMRKKARESGVYLRVVKNTLASRAVAGTEFEVVKDALVGPLLYAFSTEEPGAAGRLIKEFAKDNDKLKAKLVSVEGKLLPAAHVDVLASLPTREQALAMLARVLAEPATMFARVVKAVADKQGGGEVAAEAPAEAEPA
GCF_012999515.1_ASM1299951v1_protein.faa
>WP_111748125.1 MULTISPECIES: 50S ribosomal protein L10 [Salinisphaeraceae]
 MATSFQAKQAMVAEVGDVAKRAHSAVLAEYRGLSAGQMDELRKNAREDGAYLKVIKNTVAKRAVAGTDYECLSEALTGPVLMGFSLEDPGSAARVLTKFRNEHGVVQVTAVSLGGELLPAAQVERLASLPTYDDAISQLMRAMRGPVEALARTTREPAAKLSRSVAAVRDQKQAA
GCF_000423125.1_ASM42312v1_protein.faa
>WP_027951183.1 50S ribosomal protein L10 [Haliea salexigens]
 MAIGLDDKK

In [70]:
protein_name='30S ribosomal protein S15 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

30S ribosomal protein S15 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_010886986.1 30S ribosomal protein S15 [Deinococcus radiodurans]
 MIDKQQVISDNAAGANDTGSTAVQVALLTARINNLAGHLQTNKKDKAGQRGLQLMNGQRRRLLKYLERTDYDAYIALTDKLGIRRGQRIVR
GCF_000190555.1_ASM19055v1_protein.faa
>WP_013613829.1 30S ribosomal protein S15 [Deinococcus proteolyticus]
 MIDKKQVITDNQRGQNDTGSTEVQVALLTARINNLSAHLQANKKDKHSQRGLQLLNGQRRRLLKYMERTDYDGYIALTDKLGIRRGQRIVK
GCF_000091545.1_ASM9154v1_protein.faa
>WP_008632621.1 MULTISPECIES: 30S ribosomal protein S15 [Thermus]
 MPITKEEKQKVIQEFARFPGDTGSTEVQVALLTLRINRLSEHLKVHKKDHHSHRGLLMMVGQRRRLLRYLQREDPERYRALIEKLGIRG
GCF_000376665.1_ASM37666v1_protein.faa
>WP_013014080.1 30S ribosomal protein S15 [Meiothermus ruber]
 MPFTREEKASVIEKHAHKPGDTGSTEVQVALLTERINRLSAHLTVNKKDMAAKRGLLGLVGQRKRLLSYLESKDKARYKALIEKLGLRK
GCF_000195955.2_ASM19595v2_protein.faa
>NP_217301.1 30S ribosomal protein S15 [Mycobacterium tuberculosis H37Rv]
 MALTAEQKKEILRSYGLHETDTGSPEAQIALLTKRIADLTEHLKVHKHDHHSRRGLLLLVGRRRRLIK

GCF_000473895.1_KS51_v1_protein.faa
>WP_022607302.1 30S ribosomal protein S15 [Rubidibacter lacunae]
 MTLTQERKQQLISGFQVHETDTGSADVQVALLTARIKSLTEHLKANKKDHASRRGLLKMIGRRRRLLGFIYKEDPERYQRLIASLGIRR
GCF_000317555.1_ASM31755v1_protein.faa
>WP_015187895.1 30S ribosomal protein S15 [Gloeocapsa sp. PCC 7428]
 MALTQERKQELISNYQLHETDTGSSAVQIAMLTERINRLSEHLRSNQKDHSSRRGLLKLIGQRKRLLSYLQTEDREQYQNLISRLGIRG
GCF_001189295.1_ASM118929v1_protein.faa
>WP_050431697.1 30S ribosomal protein S15 [Chondromyces crocatus]
 MLHPVQKATLIQQYQTHEGDTGSPEVQISLLSERINQLQDHFKTHPKDHHSRRGLLKLVSQRRRQLDYLKKIDIERYRKLIGRLNLRK
GCF_000067165.1_ASM6716v1_protein.faa
>WP_012237174.1 30S ribosomal protein S15 [Sorangium cellulosum]
 MLHSVQKTDIIQKYAIHDGDTGSPEVQIALLSERINQLQEHFRTHQKDHHSRRGLLKLVSQRRRQLDYLKRVDIERYRSLINRLNLRK
GCF_002142475.1_ASM214247v1_protein.faa
>WP_017286633.1 MULTISPECIES: 30S ribosomal protein S15 [Leptolyngbya]
 MALVQERKQEIITQYQVHETDTGSADVQVAILTERINKLSLHLRSNKKDHASRTGLLKMIGQRKRLLAYIQKGDAAHYRELIARLGIRG
GCF_000013205.1_ASM

In [71]:
protein_name='30S ribosomal protein S2 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

30S ribosomal protein S2 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_010888152.1 30S ribosomal protein S2 [Deinococcus radiodurans]
 MSYISMKQLLEAGVHFGHETKRWNPKFKKFIFAERNGIFIIDLQKTLKQVDRSFDYIKDLAERGGVILFVGTKKQAQEIVELEARRTGMPYVTSRWLGGMLTNFKTMRTRIDRLNELDDLFESERINDRPKAERIQLASERERLLRFVGGIRKMNRLPDAIFVVDPTKEVIAVQEANKLGIPVIALADTDSDPDVIDYIVPGNDDAIRSIQLITHRVGDLLVEARGGHEDVSAGPVEEQSDEAQAAEQGTEGDTAQLTSSQGRS
GCF_000190555.1_ASM19055v1_protein.faa
>WP_013614151.1 30S ribosomal protein S2 [Deinococcus proteolyticus]
 MSYISMKQLLEAGVHFGHETKRWNPKFGRFIFGERNGIFIIDLQKTLKQVDRSFDYIKDLAEKGGTILFVGTKKQAQEIVELEARRTGMPFVTSRWLGGMLTNYRTIRTRVERLDELDELFESGRINDRPKAERVQLGTERDRLQRFVGGIRKMNRLPDAIFVIDPTKEVIAVKEANKLGIPVIALADTDSDPDVIDYIVPGNDDAIRSIQLITHRIGDLLVEARGADEDVEQGEERVEESNADIEAAEAGESVENVQVTSSQGRS
GCF_000091545.1_ASM9154v1_protein.faa
>WP_011172954.1 30S ribosomal protein S2 [Thermus thermophilus]
 MPVEITVKELLEAGVHFGHERKRWNPKFARYIYAERNGIHIIDLQKTMEELERTFRFIEDLAMRGGTILFVGTKKQAQDIVRMEAERAGMPYVNQRWLGGMLTNFKTISQRVHRLEELEAL

GCF_000376385.1_ASM37638v1_protein.faa
>WP_018084366.1 30S ribosomal protein S2 [Desulfurispora thermophila]
 MAVISMKQLLEAGVHFGHQTRRWNPKMAPYIFTDRNGIYIIDLQKTVRKVEEAYNFVRNLAAEGGTVLFVGTKKQAQDAVKEEAERCGMFYVNQRWLGGMLTNFQTIRRRIERLHELERMEADGSMALLPKKEVARLLHEKERLQKFLGGIKEMRRLPDALFVIDPRKEKIAVAEARKLGIPIVAIVDTNCDPDEIDYVIPGNDDAIRAVRLLTSKMADAVLEGKQGEQLSDAKPAQPAGEVAPEDSPVEAE
GCF_000233715.2_ASM23371v3_protein.faa
>WP_006522455.1 30S ribosomal protein S2 [Desulfallas gibsoniae]
 MAVISMKQLLEAGVHFGHQTRRWNPKMAPYIFTDRNGIYIIDLQKTVKKVEEAYNFVKSTVQEGGNILFVGTKKQAQESVREEAERCGMFYVNQRWLGGMLTNFQTIRRRIDRLHELEKMEQDGTMDKLPKKEVAELLHEKERLAKFLGGIKEMRRLPQALFVIDPRKERIAVAEGRKLGIPIVAIVDTNCDPDEVDYIIPGNDDAIRAVRLLTAKMADAVIEGNQGEQTAE
GCF_000315365.1_ASM31536v1_protein.faa
>WP_008413466.1 30S ribosomal protein S2 [Desulfotomaculum hydrothermale]
 MAVISMKQLLEAGVHFGHQTRRWNPKMAPYIFTDRNGIYIIDLQKTVKKVEEAYEFVKQLAAEGGSLLFVGTKKQAQEAVKEEAERCGMFYVNQRWLGGMLTNFQTIRQRIDRLHELERMEEDGTFDVLPKKEVARLLHEKEKLDKFLGGIKNMRRLPGALFVIDPRKERIAVAEARKLGIPIVA

GCF_000473895.1_KS51_v1_protein.faa
>WP_022605880.1 30S ribosomal protein S2 [Rubidibacter lacunae]
 MAVISLQELLESGVHFGHQTRRWHPRMSKYIYTARNGVHIIDLVQTAQLMEEAYSFMRDASERGERVLFVGTKRQAAGIIAQEATRCGSYFINQRWLGGMLTNWETIKTRVERLKELERLEESGALDRRPKKEAAMLRRELDKLRKYLGGIKMMRRVPDVVVIVDQRREHNAVQECRKLGIPIVSLLDTNCDPSLVDVPIPANDDAIRSVKLIVGKLADAIYAGRHGTLEGSDVGEYEEFEGGMNEYADDEGNQPSGEEE
GCF_000317555.1_ASM31755v1_protein.faa
>WP_015189294.1 30S ribosomal protein S2 [Gloeocapsa sp. PCC 7428]
 MPVVSLAQMMEAGVHFGHQTRRWNPKMAPYIYTSRNGVHIIDLVQTAQLMDEAYQYTRTAAEQGKKFLFVGTKRQAAGIIAQEAARCGSYYVNQRWLGGMLTNWTTIKTRAERLKDLERREESGALDLLPKKEASVLRREMAKLQKYLGGIKMMRKIPDVVVIVDQRREYNAVQECQKLGIQIVSMLDTNCDPDVVDVPIPANDDAIRSIKLIVGKLADAIYEGRHGQLDVEEEYDYEGAEEDLDYDESEVEYTDSLLPDEEAEPAEDEEE
GCF_001189295.1_ASM118929v1_protein.faa
>WP_050436065.1 30S ribosomal protein S2 [Chondromyces crocatus]
 MQAALTPTQNIVAPGEFPLPLRSLLDAGVHFGHQTKRWNPKMRPFIYGARNGIHIVDLDQTVRMFKRAYDFLTDAVARGGHVLFVGTKRQAQEIVQEEARRAGMYFVTNRWLGGTLTNFRTVKQGLERLRSVERMKEDGTYEQLLKKETVRLE

In [74]:
protein_name='30S ribosomal protein S3 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

30S ribosomal protein S3 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_010886962.1 30S ribosomal protein S3 [Deinococcus radiodurans]
 MGNKINPNGFRLGVTKGWNSRWYAGKKQYASLLKEDEQIRQLINKKLAAAGIARIEIERAGQQVNVIISAAKPGVVIGKGGESIKELRGDIERLVSAGTVAVNVAEIPNPNISAPLVALRIAEQIERRFAFRRAMKQAAQRVMESGARGVKVILSGRLGGAEQARTEKVLEGRVPLHTLRADIDYGTALARTTYGILGIKVLVFNGEVIGGKTETFARPQRRDRDERRPEGGDRPARRRPTARRRTGGE
GCF_000190555.1_ASM19055v1_protein.faa
>WP_013614179.1 30S ribosomal protein S3 [Deinococcus proteolyticus]
 MGNKINPNGFRLGVTRGWNSRWYAGKKNYSQMLREDEKIRQLVHKELQAAGIARIEIERAGQQINVIISAAKPGIVIGKGGESIKAIRQSIENMVEAGTVAVNVAEIPNPNTSAPLVALRIAEQIERRFAFRRAMKQAAQRVMESGARGCKVILSGRLGGAEQARTEKVLEGRVPLHTLRADIDYGTATASTTYGQLGIKVLVFNGEVIGGRTEGTQSRSARPANNEEGGERRNRGERPAGGRRRPTARRRQGGPNAAS
GCF_000091545.1_ASM9154v1_protein.faa
>WP_008633418.1 MULTISPECIES: 30S ribosomal protein S3 [Thermus]
 MGNKIHPIGFRLGITRDWESRWYAGKKQYRHLLLEDQRIRGLLEKELYSAGLARVDIERAADNVAVTVHVAKPGVVIGRGGERIRVLREELAKLTGKNVALNVQEVQNPNLSAPLVAQRVAEQIERRFAVRRAIKQAVQRVM

GCF_000376385.1_ASM37638v1_protein.faa
>WP_018084554.1 30S ribosomal protein S3 [Desulfurispora thermophila]
 MGQKVSPKGLRIGIIKDWDAKWFASKKDFSSLLVEDNKIRKYIKKKLYAAGISRVVIERAANRVKVTIHTARPGIVIGRGGAEVEALRKELEKMTGKQVHINIAEVKVPEVDAQLVAENVAAQIEKRVAFRRAMKQVVQRCLKMGGKGIKISVSGRLAGAEIARTEWYSEGKVPLHTLRADIDYGFAEADTTYGKIGIKVWIYKGEVLPEAKSAAGQGGE
GCF_000233715.2_ASM23371v3_protein.faa
>WP_006522865.1 30S ribosomal protein S3 [Desulfallas gibsoniae]
 MGQKVNPVGLRIGIIKDWEGKWYADKKNFSTLLLEDFNIRKFVKKKLYAAGIARIQIERAANKIKLSIHTAKPGIVIGRGGAEVENLRKQLEQMTGKHVNINIVEVKVPEIDAQLLSENVASQLERRIAFRRAMKQSVGRAMKMGAKGIKISCSGRLAGAEIARTEWYSEGKVPLHTLRADIDYGFAEANTTYGKIGVKVWIYKGEVLPEVRRTAAGKGGE
GCF_000315365.1_ASM31536v1_protein.faa
>WP_008410863.1 30S ribosomal protein S3 [Desulfotomaculum hydrothermale]
 MGQKVNPKGLRIGIIKDWEGKWFADKRNYSNLLIEDVKIRQFIKGKLYQAGISKTQIERAANRVKISIHTAKPGIVIGRGGAEVEALRKELEKMTGKQVHVNIVEIKNPEVDAQLVAENIAAQLEKRVAFRRAMKQTVQRSLRMGAKGIKIACSGRLAGAEIARTEWYSEGKVPLHTLRADIDYGFAEANTTYGKIGVKVWIYKGEVLPEAKKTAAGQGGE
GCF_00

>WP_015867665.1 MULTISPECIES: 30S ribosomal protein S3 [Variovorax]
 MGQKIHPTGFRLAVTRNWSSRWYASDRDFAGMLAEDIKVREYLKKKLKNASVSRVMIERPAKNARITIYSARPGVVIGKKGEDIENLKRELGKQLGVPVAVNIEEVRKPEIDAQLIADSITQQLEKRIMFRRAMKRAMQNAMRLGAQGIKIMSAGRLNGIEIARTEWYREGRVPLHTLRADIDYGTSEAKTTYGVIGVKVWVYKGDTLGRNDLPAVETPRPDDERRPRGPRRDGRPGGDRPGSDRRGPGPRAGARGPIGGNTAPADGSDKPAEATGAAPAAPGADSKPAVKRVRKAAPAAAADGAKTE
GCF_000473895.1_KS51_v1_protein.faa
>WP_022605559.1 30S ribosomal protein S3 [Rubidibacter lacunae]
 MGQKIHPTGFRLGITKEHQSCWYADSKRYPDLLQEDRKIRSYVVDNLNNAGISSIRIERKADQIDLEIHTARPGVVVGRGGSGIETLRIGLQQLLGGSRQIRINVVEVARVDADAALIGEYIAQQLERRVSFRRVVRQALQRAQRAEVKGIKIQIGGRLNGAEIARTEWTREGRVPLHTLRADIDYAYRTARTIYGILGIKIWVFKGEIIPGADGPPPPPPAPRRQRRRQFEDRSGE
GCF_000317555.1_ASM31755v1_protein.faa
>WP_015187436.1 30S ribosomal protein S3 [Gloeocapsa sp. PCC 7428]
 MGQKIHPVGFRLGVTQEHQSRWFAEPARYPEILKEDHKLRNYVEQKLGRLAQNNAGISEVRIERKADQIDLEVRTARPGVVVGRGGTGIESLRTGLQELLGGNRQIRINVVEVQQVDADAYLIAEYIAQQLERRVSFRRVVRQTIQRAQKAGIQGIRIQVSGRLNGAEIARTEWTR

In [79]:
protein_name='50S ribosomal protein L18 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

50S ribosomal protein L18 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_010888743.1 50S ribosomal protein L18 [Deinococcus radiodurans]
 MATATTIRRKLRTRRKVRTTTAASGRLRLSVYRSSKHIYAQIIDDSRGQTLAAASSAALKSGNKTDTAAAVGKALAAAAAEKGIKQVVFDRGSYKYHGRVKALADAAREGGLDF
GCF_000190555.1_ASM19055v1_protein.faa
>WP_013614189.1 50S ribosomal protein L18 [Deinococcus proteolyticus]
 MSNTSATRRKLRARRKVRTAMGERPRLSIYRSSKHIYAQIIDDKSGTTLAAASTGALKEAAAGSKTDRAAAVGKALAEAAAAKGVKQVAFDRGQYRYHGRVKALADAAREGGLDF
GCF_000091545.1_ASM9154v1_protein.faa
>WP_008633391.1 MULTISPECIES: 50S ribosomal protein L18 [Thermus]
 MARLTAYERRKFRVRNRIKRTGRLRLSVFRSLKHIYAQIIDDEKGVTLVSASSLALKLKGNKTEVARQVGRALAEKALALGIKQVAFDRGPYKYHGRVKALAEGAREGGLEF
GCF_000376665.1_ASM37666v1_protein.faa
>WP_013014851.1 50S ribosomal protein L18 [Meiothermus ruber]
 MARLTTEDRRKFRVRNAVKASGRLRLSVHRSLQHIYAQIIDDSKGHTLVATSSKSLKLSGNKTEVARKVGLAIAEAAKAKGISQVVFDRGAFKYHGRVKALAEGAREGGLEF
GCF_000195955.2_ASM19595v2_protein.faa
>NP_215234.1 50S ribosomal protein L18 [Mycobacter

GCF_000170895.1_ASM17089v1_protein.faa
>WP_006974405.1 50S ribosomal protein L18 [Plesiocystis pacifica]
 MAKKNKIQARLRRKKSIRRRITGSAERPRMSVFRSNKHIYVQVIDDDAAKSLLTVSDLGKDLNTELEACESKVDRAKVVGRAAAKACLDKGITKVVFDRNGYLYHGRVSALADAAREAGLQF
GCF_001273775.1_ASM127377v1_protein.faa
>WP_083447772.1 50S ribosomal protein L18 [Nitrospira moscoviensis]
 MNTAEKERQLERRRRRVRQRIMGTPERPRLNVFRSSAHIYAQIIDDIKGTTLAAASSLDKSLRTSLKSTSSIEGAKAVGKLLADRAKAAKITAVVFDRGGRMYHGRIKALAEASREGGLQF
GCF_000025705.1_ASM2570v1_protein.faa
>WP_041421025.1 50S ribosomal protein L18 [Sideroxydans lithotrophicus]
 MNKNEARQRRARKTRARIAEQKTVRLAIHRTNLHIYAQVISADGGKVLASASTLEAEVRKSVGNGGNAAAAAVVGKRIAEKAKSAGVTAVAFDRSGNKYHGRIKALADAAREHGLQF
GCF_004365595.1_ASM436559v1_protein.faa
>WP_066805530.1 50S ribosomal protein L18 [Pseudodesulfovibrio indicus]
 MSISKNEQRLRRKPRIRKKISGTEARPRLVVYRSNVHLYAQLVDDVNGVTLASASTQVLNKGGEALKANKDSAAKVGKAIAEAALAQKIETCVFDRNGYIYHGKIKALADGAREGGLKF
GCF_000266945.1_ASM26694v1_protein.faa
>WP_014812967.1 50S ribosomal protein L

In [112]:
protein_name='30S ribosomal protein S14 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

30S ribosomal protein S14 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_027479943.1 30S ribosomal protein S14 [Deinococcus radiodurans]
 MANKSKLAKQKQREKTVEKYAAKRAELKAAGDYYGLTQLPRDASPTRLHNRCEFTGRPRGYVRFFGVSRIVLREMAHRGELPGVKKASW
GCF_000190555.1_ASM19055v1_protein.faa
>WP_013614186.1 30S ribosomal protein S14 [Deinococcus proteolyticus]
 MAKKSKIAKQKQREQKVAKYAAKRAEMKAAGDYYGLSQLPRDASPTRLHNRCEFTGRPRGYIRHFGVSRIVMREMAHRGELPGVRKASW
GCF_000091545.1_ASM9154v1_protein.faa
>WP_008633399.1 MULTISPECIES: type Z 30S ribosomal protein S14 [Thermus]
 MARKALIEKAKRTPKFKVRAYTRCVRCGRARSVYRFFGLCRICLRELAHKGQLPGVRKASW
GCF_000376665.1_ASM37666v1_protein.faa
>WP_013014854.1 MULTISPECIES: 30S ribosomal protein S14 [Meiothermus]
 MAKKSQVEKMKRKLKTIAKYAAKRAALKAAGDYAALAELPRDASPTRHKNRCAVTGRSKAYMRYFGLSRLQFREMAHKGQLPGVKKASW
GCF_000195955.2_ASM19595v2_protein_EDIT_RM_S14.faa
>NP_216572.1 30S ribosomal protein S14 [Mycobacterium tuberculosis H37Rv]
 MAKKSKIVKNQRRAATVARYASRRTALKDIIRSPSSAPEQRSTAQRALARQPRDASPVRLRNRDAIDGRPRGH

>WP_015867798.1 MULTISPECIES: 30S ribosomal protein S14 [Variovorax]
 MAKQSLIQRELKRDKLVAKYAAKHAELKAISNDLKKSDEERAAARLGLQKLPRNANPTRQRNRCEITGRPRGTFRQFGLARAKIRELAFNGDIPGVTKASW
GCF_000473895.1_KS51_v1_protein.faa
>WP_022606567.1 30S ribosomal protein S14 [Rubidibacter lacunae]
 MAKKSMIARERKREKLVAKYSAKREELKEQIRTAQTPLERFELRRKLQNLPRNSSPTRLHNRCKVTGRPRGYYRDFGLSRNVLREWAHEGLLPGVTKSSW
GCF_000317555.1_ASM31755v1_protein.faa
>WP_015189741.1 MULTISPECIES: 30S ribosomal protein S14 [Chroococcaceae]
 MAKKSMIEREKKRQKLVTKYAQKREALIEQFEQATSQREKLEIHRQIQQLPRNSAPTRLHNRCWVTGRPRGVYRDFGLSRHVLREWAHQGLLPGVVKSSW
GCF_001189295.1_ASM118929v1_protein.faa
>WP_082362215.1 type Z 30S ribosomal protein S14 [Chondromyces crocatus]
 MARAKEFAKLNRPPKFAVRHRNRCKVCGRSRAYYRDFELCRVCLRLFALRGELPGVIKASW
GCF_000067165.1_ASM6716v1_protein.faa
>WP_012240559.1 type Z 30S ribosomal protein S14 [Sorangium cellulosum]
 MARAKEFAKLNRPPKFSTRHKNRCKVCGRARAYYRDFELCRVCLRLFALRGELPGVIKASW
GCF_002142475.1_ASM214247v1_protein.faa
>WP_026148789.1 MULTIS

In [113]:
checkGenome('GCA_000018565.1_ASM1856v1_protein_EDIT_ADD_L29.faa')

50S ribosomal protein L1 [
>ABX02775.1 50S ribosomal protein L1 [Herpetosiphon aurantiacus DSM 785]
 MAKQHGKKYLAAAAKVEADRLYTPEEAAALVKETSFTNFDATVELHVRLGIDPRHADQTVRSTASLPHGTGKSVRVLVFAQGDTENIARNAGAEIVGGDELIQRIEKENFIDFDVAVATPDMMGKVGRLGRILGRRGLMPNPKSGTVVPPGDLPQAIKALQGGRVEFRNDKTGLLHVAIGKISFTPDQIRENLVVLLDAIKAAKPSGSKGVYIKSLTLTSTMGPGIPLDVNVAQATA
1
50S ribosomal protein L2 [
>ABX07548.1 50S ribosomal protein L2 [Herpetosiphon aurantiacus DSM 785]
 MGIKRYKPTSPGRRGMTVSDFAEITKFEPEKSLTEPLKKHAGRNNHGHITTRHRGGGHKRRYRLIDFKRKKFDIPAKVAGIEYDPNRSANIALLHYTDGEKRYIIAPLGLKVGDTLMAGPNAEIRVGNALPLANIPIGSQIHNIELTPGRGGQLVRSAGNTAQLMAREGNYATVRLPSGEMRMVHIKCMATLGQVGNVDHQNIMIGKAGRSRWLGRRPVVRGSAMNPVDHPHGGGEGRAPTGMNPKTKWGKPAMGKKTRHNPRTQRFIVRTRKQK
1
50S ribosomal protein L3 [
>ABX07545.1 50S ribosomal protein L3 [Herpetosiphon aurantiacus DSM 785]
 MLNGILGRKIGMTQIFTEKGETIPVTVIEAGPCVVTQLRTKDKDGYEAVQLGFGEIKPRKVTKPIQGHLKAAGRLVRFMREVKTTDLNAHNVGDVVNVDIFQIGEKIDVVGTSKGRGFAGVVKRHGFRGGPATHGQSDRHRAPGSIGSGTTPGRVWKNMRMAGRMGNDRVTVQNLEVVK

In [114]:
checkGenome('GCF_000013425.1_ASM1342v1_protein.faa')

50S ribosomal protein L1 [
>YP_499092.1 50S ribosomal protein L1 [Staphylococcus aureus subsp. aureus NCTC 8325]
 MAKKGKKYQEAASKVDRTQHYSVEEAIKLAKETSIANFDASVEVAFRLGIDTRKNDQQIRGAVVLPNGTGKSQSVLVFAKGDKIAEAEAAGADYVGEAEYVQKIQQGWFDFDVVVATPDMMGEVGKLGRVLGPKGLMPNPKTGTVTMDVKKAVEEIKAGKVEYRAEKAGIVHASIGKVSFTDEQLIENFNTLQDVLAKAKPSSAKGTYFKSVAVTTTMGPGVKIDTASFK
1
50S ribosomal protein L2 [
>YP_500976.1 50S ribosomal protein L2 [Staphylococcus aureus subsp. aureus NCTC 8325]
 MAIKKYKPITNGRRNMTSLDFAEITKTTPEKSLLKPLPKKAGRNNQGKLTVRHHGGGHKRQYRVIDFKRNKDGINAKVDSIQYDPNRSANIALVVYADGEKRYIIAPKGLEVGQIVESGAEADIKVGNALPLQNIPVGTVVHNIELKPGKGGQIARSAGASAQVLGKEGKYVLIRLRSGEVRMILSTCRATIGQVGNLQHELVNVGKAGRSRWKGIRPTVRGSVMNPNDHPHGGGEGRAPIGRPSPMSPWGKPTLGKKTRRGKKSSDKLIVRGRKKK
1
50S ribosomal protein L3 [
>YP_500979.1 50S ribosomal protein L3 [Staphylococcus aureus subsp. aureus NCTC 8325]
 MTKGILGRKIGMTQVFGENGELIPVTVVEAKENVVLQKKTVEVDGYNAIQVGFEDKKAYKKDAKSNKYANKPAEGHAKKADAAPKRFIREFRNVDVDAYEVGQEVSVDTFVAGDVIDVTGVSKGKGFQGAIKRHGQSRGPMSHGS

In [75]:
checkGenome('GCF_012911975.1_ASM1291197v1_protein.faa')

50S ribosomal protein L1 [
>WP_169611732.1 50S ribosomal protein L1 [Nodosilinea sp. P-1105]
 MTKTISKRLRDAQAKVEDRLYEPLEALQLLKDTATAKFTESAEAHIRLGIDPKYTDQQLRTTVILPKGTGQTVRVAVIAKGEKVAEANAAGADLVGSEELIAEIQKGMMDFDVLIATPDMMPQVAKLGRQLGPRGLMPSPKGGTVTFDLPQAIEEFKAGKLEFRADKTGIVHVMFGKVDFSIEDLLVNLKALQDCIDRNRPSGAKGRYWRSVTVSSTMGPGIPVDVSALRDLKLDAVAA
1
50S ribosomal protein L2 [
>WP_169611826.1 50S ribosomal protein L2 [Nodosilinea sp. P-1105]
 MGIRSYRPYTPGTRERTVSEFAEITRSEPEKSLTRSKHRPKGRNNRGVITCRHRGGGHKRLYRVVDFHRNKVGVSAKVASIEYDPNRNARISLLHYADGEKRYILCPAGLTVGATVVSGPDAPLEVGNALPLYKIPLGTTVHNIEMQAGKGGQMARSAGAGAQVVAKEGDYVTLKLPSTEVRMVRRECYATIGQVGNADIRNVSLGKAGRKRWMGRRPEVRGSVMNPVDHPHGGGEGRAPIGRSGPVTPWGKPTLGYKTRKKNKDSDKYVVRRRRRVSKRGRGGRNA
1
50S ribosomal protein L3 [
>WP_206069945.1 50S ribosomal protein L3 [Nodosilinea sp. P-1105]
 MTQIFDQAGNAVPVTVVEAGPCVVTQIKTPDTDGYAAIQLGFGEATEKALNKPKLGHLSRSGSTPVRHLKEYRVEAADAYELGQAITADSFESGQLVDISGKSIGKGFAGYQKRHNFRRGPMSHGSKNHRQPGSTGAGTTPGRIYPGKRMAGQMGNAQVTIRKLEVVRVDSDRNLLLIKGAIPGKP

In [115]:
#checkGenome('GCF_001273775.1_ASM127377v1_protein.faa') #missing L10 Nitrospira m.
checkGenome('GCF_001273775.1_ASM127377v1_protein_EDIT_ADD_L10.faa')

50S ribosomal protein L1 [
>WP_053378937.1 50S ribosomal protein L1 [Nitrospira moscoviensis]
 MGKKMKAALEKVEPRAYALREAVDVVKKAAYAKFDESVDLALRLGIDPKRSDQMVRGTTSLPHGTGKQVRVVVFAKGEKEQEARQAGADFVGADDLMEKIKGGWMDFDYAIATPDLMASVGKLGKQLGPRGLMPNPKTGTVTFEVGKAVGEIKKGRVEFKVEKAGIVQVPVGKVSFDVNKLYDNAAAIIESVIKAKPASCKGRYLKSATISSTMGPGVQLDTVALTKQWS
1
50S ribosomal protein L2 [
>WP_053378945.1 50S ribosomal protein L2 [Nitrospira moscoviensis]
 MGLKSYRPTSAGRRGMTAVTTEELTKKKPEKSLTSFHVRTGGRNNDGRMTIRFRGGGHKRLYRTIDFRRDKAGIPARVEAIEYDPNRSARIALLKYRDGEKRYILAPVGLNVNDEVQSGPEAEIRPGNALALSSMPLGTTIHNIELKPGKGAQLIRSAGGFAQVMGREGGYVQVRLKSGEMRRILGSCMATVGQVGNLDHENVSVGKAGRSRWKGKRPHVRGVVMNPVDHPHGGGEGKSGQGNPHPVSPWGLPTKGYKTRQNKKTDKYIIARRKSGVRNA
1
50S ribosomal protein L3 [
>WP_053378943.1 50S ribosomal protein L3 [Nitrospira moscoviensis]
 MTNGLLGKKLGMTQIYDESRLTPVTVIEAGPCRVVTVKTKERDGYEAVQLSFGEVKERKLSKAELGHLKKQQAPASRWLREFKKDGEVAVGQSVTVGIFKKGDWVDVIGVSKGKGFQGVVRRHHYAGGPESHGSMFHRAPGSIGSSSFPSRVWKGKTLPGHMGAERVTAQRLKVVESRPEENLLFVRGAVPGAANGLIM

In [116]:
#checkGenome('GCF_000710505.1_Plagard2.0_protein.faa') #Planktothrix a. #missing L23
checkGenome('GCF_000710505.1_Plagard2.0_protein_EDIT_ADD_L23.faa')

50S ribosomal protein L1 [
>WP_042156740.1 50S ribosomal protein L1 [Planktothrix agardhii]
 MARQESRRLRELRKKVEDRPYEPLEAINLLKETATAKFPESAEAHIRLGIDPKYTDQQLRTTVSLPKGTGQVIRVAVLAKGEKVTEAQAAGADVVGSEDLINDIQKGMLDFDCLIATPDMMPQVAKLGRLLGPKGLMPSPKGGTVTTDVAKAIAEFKAGKLEFRADRTGIVHVMFGKTAFSAADLLVNLSTLQECIDRNRPSGAKGRYWRSVFVASTMGPSIEVDITALRELKIADIV
1
50S ribosomal protein L2 [
>WP_042151659.1 MULTISPECIES: 50S ribosomal protein L2 [Planktothrix]
 MGIRSYRPYTPGTRQHTVSDFAEITRSEPERSLTTSKHRDKGRNNRGVITCRHRGGGHKKLYRIVDFRRDKHEILAKVAEIEYDPNRNARIALVYYTDGEKRYILHPLGLKVGTMIVSGPEAPIEIGNALPLSKIPLGTDVHNVELIPGKGGQIVRAAGTTAQVLAKEGDYVTLKLPSGEQRKIPAKCYATIGQVGNIDARNISIGKAGRKRWKGRRPEVRGSVMNPVDHPHGGGEGRAPIGRSGPVTPWGKPALGAKTRNKKKRSTALIVRRRRKSSKRGKGGRQT
1
50S ribosomal protein L3 [
>WP_026797651.1 MULTISPECIES: 50S ribosomal protein L3 [Planktothrix]
 MSVGILGTKVGMTQIFEAESGKAIPVTVIQAGPCIVTQIKTKQTDGYTAIQLGYGEVPDNKRKLNTKKTPDKEVNKYLSNAEQGHLAKSGGTCVRHLKEYRVSEPESFQLGQQLKADIFTQGQLVDVSGTSMGRGFSGYQKRHNFKRGPMAHGSKNHRQPGSIGPGTTPGRVYPGKRMAG

In [117]:
checkGenome('GCF_000196815.1_ASM19681v1_protein.faa')

50S ribosomal protein L1 [
>WP_013247877.1 50S ribosomal protein L1 [Nitrospira defluvii]
 MGKKMTAAQEKVEPRFYGLKEAVEAVKQAAFAKYDESVDLAIRLGIDPKRSDQMVRGTTALPHGTGKKLRVLVFAKGEKEQEARQAGADHVGSDDLMEKIKAGWMEFDCAISTPDLMASVGKLGKVLGPRGLMPNPKTGTVTFEVGKAVSEIRKGRVEFKVEKAGIVQVPVGKVSFDVQKLYDNAQAVLESVVKAKPSSCKGRYIKSVTMSSTMGPGVKLDPVALSKLWS
1
50S ribosomal protein L2 [
>WP_013247889.1 50S ribosomal protein L2 [Nitrospira defluvii]
 MALKVYKPTTPGRRGMTAIQGEKLTKKKPEKALTGFHLRTGGRNNDGRQTIRFRGGGHKRLYRQIDFRRDKAGIPAKVASIEYDPNRSSRIALLHYVDGEKRYILAPVGLSINDVVQSGVGAEVRPGNALPLVNIPLGTTIHNIELKPGKGGQLIRSAGGSAQVMGRDGAYVQIRLKSGEMRKVLSTCMATVGQVGNVDHENVVVGKAGRTRWKGKRPHVRGVVMNPVDHPHGGGEGKSGQGNPHPVSPWGTPAKGYKTRKNKATDKFIISRRKK
1
50S ribosomal protein L3 [
>WP_013247886.1 50S ribosomal protein L3 [Nitrospira defluvii]
 MTNGLLGKKLGMTQVYDESVLAPVTVIEAGPCRVVSVKTKERDGYEAVQLSFGEVKEQRLSRGELGHLKKHQAAPSRWLREFPKVGDVTVGQTVKVDMFKKGDWVDVVGVSKGKGFQGVVRRHNYAGGPETHGSMFHRAPGSIGASSYPSRVWKNKALPGHMGDERVTVQRLKVVDARPDENLLFVRGAVPGGVNGLLIVRKSKKS
1
50S rib

In [ ]:
#L5,L22,L24,S4,S5

In [119]:
protein_name='50S ribosomal protein L5 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

50S ribosomal protein L5 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_010886968.1 50S ribosomal protein L5 [Deinococcus radiodurans]
 MQQLKTKYNDQVRPALMQQFGYSSVMAVPRIEKIVVNEGLGSSKEDSKAIDKAAKELALITLQKPIITKAKKSISNFKLRQGMPVGIKVTLRGERMYVFLEKLINIGLPRIRDFRGINPNAFDGRGNYNLGIKEQLIFPEITYDMVDKTRGMDITIVTTAKTDEEARALLQSMGLPFRKQ
GCF_000190555.1_ASM19055v1_protein.faa
>WP_013614185.1 50S ribosomal protein L5 [Deinococcus proteolyticus]
 MLQLKEKYQSEVRPALVEQFSYSSVMAAPRIEKIVINEGLGSAARDDSKAIDKAAKELALITLQKPVITKAKKSISNFKLRQGMPIGVKVTLRGDRMWVFLDKLINVGLPRIRDFRGINPNAFDGRGNYNLGIKEQLIFPEITYDMVDRTRGMDITIVTTAKTDEEARALLQGLGLPFRR
GCF_000091545.1_ASM9154v1_protein.faa
>WP_011228843.1 MULTISPECIES: 50S ribosomal protein L5 [Thermus]
 MPLDVALKRKYYEEVRPELIRRFGYQNVWEVPRLEKVVINQGLGEAKEDARILEKAAQELALITGQKPAVTRAKKSISNFKLRKGMPIGLRVTLRRDRMWIFLEKLLNVALPRIRDFRGLNPNSFDGRGNYNLGLREQLIFPEITYDMVDALRGMDIAVVTTAETDEEARALLELLGFPFRK
GCF_000376665.1_ASM37666v1_protein.faa
>WP_013014855.1 MULTISPECIES: 50S ribosomal protein L5 [Meiothermus]

>WP_006974408.1 50S ribosomal protein L5 [Plesiocystis pacifica]
 MAKAAATQKKKPEAGYAFKFPKVRNEPVDRGDFVPHTRTLYREKVIPALKEEFGYKNPNTIPTVEKVVLNIGLGEAIKQPKLLEQSFECLGNITGQRPVVTRAKKSIAGFKLREGMKIGAMVTLRRTAMWDFLDRLLMVALPRTRDFRGVSRKAFDGRGNYTLGIKEILIFPEVDIEKLNAVPGMNVCIHTSAKTDDEARALLKHLGMPFRQ
GCF_001273775.1_ASM127377v1_protein_EDIT_ADD_L10.faa
>WP_053382003.1 50S ribosomal protein L5 [Nitrospira moscoviensis]
 MYQQQVVPALMKEFGYKNVMQVPKLERVVLNVGMGEAIQNVKLLESAVAELGTITGQKPVVTRAKKAIAGFKLRQGLPIGAKVTLRSRRMYEFFDRLVTLALPRIRDFRGVSPKAFDGRGNYTLGLKEQLIFPEIKYDEVASIHGMDITIVTTARTNDEGKALLKHLGMPFRT
GCF_000025705.1_ASM2570v1_protein.faa
>WP_013028916.1 50S ribosomal protein L5 [Sideroxydans lithotrophicus]
 MARLYDYYKKNVTEQLMKQFGYKSVMEVPRIEKITLNMGVGEAVADKKVMDHAVGDMQKIAGQKPVVTKSKKSIAGFKIRENYPVGCKVTLRRERMYEFLDRLVTVAIPRIRDFRGISGKSFDGRGNYNMGVKEQIIFPEIEYDKIDALRGMNITITTTAKTDEEAKALLLAFKFPLKN
GCF_004365595.1_ASM436559v1_protein.faa
>WP_066805535.1 50S ribosomal protein L5 [Pseudodesulfovibrio indicus]
 MTRLEKVYNEKVVPELQKEYGYSS

GCF_000346485.2_ASM34648v2_protein.faa
>WP_017748881.1 50S ribosomal protein L5 [Scytonema hofmannii]
 MAAPRLKTVYQETIVPKLMKQFEYINIHQVPKLVKVTVNRGLGEASQNAKALEASLSEIALITGQKPVVTRAKKAIAGFKIRQGMPVGIMVTLRGERMYAFLDRLINLTLPRIRDFRGISPKSFDGRGNYTLGVREQLIFPEVEYDSIDQVRGMDISIITTAKNDEEGRALLKEMGMPFREQ
GCF_001617355.1_Rhodanobacter_sp._FW104-T7_protein.faa
>WP_185754695.1 MULTISPECIES: 50S ribosomal protein L5 [Rhodanobacter]
 MMTRLENFYKEQVVPKLTKQFGYQNVMQVPRITKITLNMGVGEAAGNKKVLENAVADMAKISGQKPITTKARVSVASFKIRDGWPIGCKVTLRRAQMFEFLDRLINISLPRVRDFRGVSGKAFDGRGNYNFGIKEQIIFPEIDFDQVDALRGMDISITTTAKTDEEAKALLAAFSFPFRN
GCF_012999515.1_ASM1299951v1_protein.faa
>WP_111748144.1 MULTISPECIES: 50S ribosomal protein L5 [Salinisphaeraceae]
 MSEQKQWSPRLQRTYRETIVPRLIEQFGYTNTMAVPALSKITLNMGLGEAVSDRKVIDSAMEELGLIAGQKPVMTRARKSEAGFKIRTGWPIGCKVTLRRARMYEFLDRLVNVALPRIRDFRGLSPKSFDGRGNYSLGVREQIVFPEIDYDRVDTLRGMDIVITTTAHNNAEARGLLDGFELPFRNS
GCF_000423125.1_ASM42312v1_protein.faa
>WP_027950930.1 50S ribosomal protein L5 [Haliea salexigens]
 

In [136]:
protein_name='50S ribosomal protein L24 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

50S ribosomal protein L24 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_187767769.1 50S ribosomal protein L24 [Deinococcus radiodurans]
 MHFKKGDTVIVLSGKHKGQTGKVLLALPRDQKVVVEGVNVITKNVKPSMTNPQGGQEQRELALHASKVALVDPETGKATRVRKQIVDGKKVRVAVASGKTID
GCF_000190555.1_ASM19055v1_protein.faa
>WP_174260423.1 50S ribosomal protein L24 [Deinococcus proteolyticus]
 MHVRKGDTVVVLSGKHKGATGKVLETSPRDHKVLVEGVNTVIKHVKPSFANPEGGRVEKEIALHASKVALVDPETGKATRVRKQIVDGRKVRVAAASGKVID
GCF_000091545.1_ASM9154v1_protein.faa
>WP_011173707.1 50S ribosomal protein L24 [Thermus thermophilus]
 MRVKMHVKKGDTVLVASGKYKGRVGKVKEVLPKKYAVIVEGVNIVKKAVRVSPKYPQGGFIEKEAPLHASKVRPICPACGKPTRVRKKFLENGKKIRVCAKCGGALDTEE
GCF_000376665.1_ASM37666v1_protein.faa
>WP_013014856.1 MULTISPECIES: 50S ribosomal protein L24 [Meiothermus]
 MKVHVKKGDTVIVLSGKEGLRGETGKVKAVMPKEGLVVVEGLNLIKRAVRPNPRNPQGGFIETEAPIHASKVMPICPSCEKPTRIRKKVLPDGTKVRACAKCDGVLDTK
GCF_000195955.2_ASM19595v2_protein_EDIT_RM_S14.faa
>NP_215229.1 50S ribosomal protein L24 [Mycobacterium tubercu

>WP_053378952.1 50S ribosomal protein L24 [Nitrospira moscoviensis]
 MGDVVQALGKSRIRKGDTVVVIAGRERGKSGKVLSVDLGAGKVTVEKLNIIKRHTKPNQKAKQGGILEREAPLAISNVMFLCPVTQKPTRMGIRTLADGRRVRFSKKSNETIE
GCF_000025705.1_ASM2570v1_protein.faa
>WP_013028915.1 50S ribosomal protein L24 [Sideroxydans lithotrophicus]
 MNKIKKNDDVIVIAGKDKGNRGSVLNVLGNDRVLVSGVNKVKKHQKPNPMKGTTGGIVEIEKPIHVSNIAIYNSATKKADRVGVKQLEDGRKVRVFKSSGEVIA
GCF_004365595.1_ASM436559v1_protein.faa
>WP_066805538.1 50S ribosomal protein L24 [Pseudodesulfovibrio indicus]
 MMKTKIRKDDKVMVIAGKDKGKVGKVLKILKKQDKVLVEKVNMVQRHTKANPYAQQPGGIIEKEAPIHVSNVAVVCDACTKPTRVGYKKTEDGKKVRFCKKCNETFK
GCF_000266945.1_ASM26694v1_protein.faa
>WP_014812962.1 50S ribosomal protein L24 [Desulfomonile tiedjei]
 MGNIKKNDKVKVISGKEKGRQGKILKVLGEKDAALVERLNLVKRHTKAGRAGQQGGIVEKEAPIKLSKLMLVCPKCSKPTRTGSRVLDDGKRVRYCKKCSEQLES
GCF_000014965.1_ASM1496v1_protein.faa
>WP_011698423.1 50S ribosomal protein L24 [Syntrophobacter fumaroxidans]
 MKGQKSYRIKKNDTVMVVTGKDKGKSGKVLRVIYKKDRAIVEKLNMIKRHMKPSQQ

In [128]:
protein_name='50S ribosomal protein L22 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

50S ribosomal protein L22 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_010886961.1 50S ribosomal protein L22 [Deinococcus radiodurans]
 MTAPEQTFRNKKQRKQQVKLRKPGFAVAKYVRMSPRKVRLVVDVIRGKSVQDAEDLLRFIPRSASEPVAKVLNSAKANALHNDEMLEDRLFVKEAYVDAGPTLKRLIPRARGSANIIKKRTSHITIIVAEKGNK
GCF_000190555.1_ASM19055v1_protein.faa
>WP_013614178.1 50S ribosomal protein L22 [Deinococcus proteolyticus]
 MTATQTAEFRNKKQRKQEHKLRKPGFARAKYVRMSPRKVRLVVDLIRGKSVRDAEDILRFLPHIASEPISKVLNSAKANALHNDDMLEDQLFIKEAYVDAGPTLKRLIPRARGAANIIRKRTSHITIVVGEKGAN
GCF_000091545.1_ASM9154v1_protein.faa
>WP_011173710.1 MULTISPECIES: 50S ribosomal protein L22 [Thermus]
 MEAKAIARYVRISPRKVRLVVDLIRGKSLEEARNILRYTNKRGAYFVAKVLESAAANAVNNHDMLEDRLYVKAAYVDEGPALKRVLPRARGRADIIKKRTSHITVILGEKHGK
GCF_000376665.1_ASM37666v1_protein.faa
>WP_041654122.1 50S ribosomal protein L22 [Meiothermus ruber]
 MATARYVRMSPRKVRLVVDLIRGKKLEEARAILKYTPHRASEVVAKVLESAAANGMNDDRKNMIEDQIYVKAAYVDEGPAIKRVLPRARGSANMIKKRTSHITIIVGEKNG
GCF_000195955.2_ASM19595v2_protein_EDIT_RM_S14.fa

>WP_053378947.1 50S ribosomal protein L22 [Nitrospira moscoviensis]
 MAEAKAILRFVRVAPRKAKPVIDMIRGQQVPMALAMLKHTPRHAARVVEKILKSAVANAEQKEMGDSESMVISKAFVDCGPTYKRFRARSMGRANAIQKRTSHITVVVAAPSLDGKRK
GCF_000025705.1_ASM2570v1_protein.faa
>WP_190272146.1 50S ribosomal protein L22 [Sideroxydans lithotrophicus]
 MMSTTAVIRGVRLSAQKARLVADQIRGLPVAQALNILAFSPKKGGGIIKKALDSAIANAEHNDGADIDELKVATIYVDKGTSLKRMIPRAKGRGNSIEKQTCHITITVGS
GCF_004365595.1_ASM436559v1_protein.faa
>WP_066805560.1 50S ribosomal protein L22 [Pseudodesulfovibrio indicus]
 MEAKAVAKFIRVSPRKTRVVAENIKGKGVEDALNILRFTPKKPAEILRKVLYSAISNAEQMPGVDVDSLIVDTVMVNEGPTWKRIKPRAMGRAYRIRKRTSHITIVVKEQ
GCF_000266945.1_ASM26694v1_protein.faa
>WP_014812956.1 50S ribosomal protein L22 [Desulfomonile tiedjei]
 MEWVSTHKYARISARKARLVADLVRGMPVSKAMSLLRFTPKKGAELILKVVQSALGNATQSRGVDEESLYISRIMIDEGPTAKRWIPRAMGRATRVRKRTSHIVVAVSEE
GCF_000014965.1_ASM1496v1_protein.faa
>WP_011698417.1 50S ribosomal protein L22 [Syntrophobacter fumaroxidans]
 MEVRAVSKFLRVSPHKARLVADLVRGKKVSDALTILK

In [137]:
protein_name='30S ribosomal protein S4 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[227:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

30S ribosomal protein S4 [
GCF_008329785.1_ASM832978v1_protein.faa
>WP_010888758.1 30S ribosomal protein S4 [Deinococcus radiodurans]
 MGRFRGSITKQSRREGINLAETEKVQKYLDRRPYGPGQHGQRRKGRPSDYSVRLREKQKLARLYGMNEKQFRNLFEEASNVPGVTGTVFLQLLERRLDNVVFRMGFASTRRQARQFVGHGHILVNGKKVDIASYRVKIGDEISVAEGSRSMGFIQENMDAQKRRRVSPWIEMDQDNFKGTFSRLPAREDLALPINENFIIEYYSR
GCF_000190555.1_ASM19055v1_protein.faa
>WP_013614200.1 30S ribosomal protein S4 [Deinococcus proteolyticus]
 MGRFSGSIVKQSRREGINLAETEKVQKYLDRRPYAPGQHGQRRGRGRPSDYSVRLREKQKLSRLYGMNEKQFRNLFEEATRQPGVLGTVFLQLLERRLDNVVFRMGFASTRRQARQFVGHGHILVNGKKVDIASYRVKIGDEITVAEKSRSMGFVQENMEAMKRRRPSPWLEVNAETFTGTFNRLPAREDLALPINENFIIEYYSR
GCF_000091545.1_ASM9154v1_protein.faa
>WP_011173699.1 30S ribosomal protein S4 [Thermus thermophilus]
 MGRYIGPVCRLCRREGVKLYLKGERCYSPKCAMERRPYPPGQHGQKRARRPSDYAVRLREKQKLRRIYGISERQFRNLFEEASKKKGVTGSVFLGLLESRLDNVVYRLGFAVSRRQARQLVRHGHITVNGRRVDLPSYRVRPGDEIAVAEKSRNLELIRQNLEAMKGRKVGPWLSLDVEGMKGKFLRLPDREDLALPVNEQLVIEFYSR
GCF_000376665.1_ASM37666v1_pro

GCF_001273775.1_ASM127377v1_protein_EDIT_ADD_L10.faa
>WP_053378965.1 30S ribosomal protein S4 [Nitrospira moscoviensis]
 MAKYRGPVCRLCRREGEKLFLKGSRCMTEKCAIERRSYPPGQHGQGRPRTSDYSLQLREKQKLRRIYGLQERQFRSVFERAERQSGVTGDALLRLLECRLDNVAYRLGFAASRKQARQIVSHGHLTLNGKKITVAGALVKAGDVIEVRERSRNLVAIQSALEAVDGRGVPEWLELDKGAFKGTVRALPAKEHIALPVNEQMVVELYSR
GCF_000025705.1_ASM2570v1_protein.faa
>WP_013028929.1 30S ribosomal protein S4 [Sideroxydans lithotrophicus]
 MARNLDAKCRQCRREGEKLFLKAEKCFTDKCSVERRAYAPGQHGQKKNQRQSEYGGQLREKQKLRRIYGMMEGQFRLTYKRAEQSRGITGESLLQMLESRLDNVSYRMGFGASRSEARQVVRHNSILVNGKRVNIPSYQVRPGDVVEVAEKSRNQLRVKAALQAAEQRGFPEWLEVDAKAFKGTFKAVPQRSELPATINEHLVVELYSK
GCF_004365595.1_ASM436559v1_protein.faa
>WP_066805511.1 30S ribosomal protein S4 [Pseudodesulfovibrio indicus]
 MARYTEAKCKLCRREGEKLFLKGDRCYTDKCAYEKRPYPPGHSGRMRHKMSDYAIQLREKQKVRRMYGVLEGQFREYYHRADGMKGVTGHNLLFLLERRLDNVIYRLGFANSRDQARQLVRHGIFKLNGRRVNIPSMQVKAEDVIEVREEARKIPVINEAQEVIARRGCPEWLESDGANFKGTVKAMPSRDDIQFPINEQLIVELYSK
GCF_000266945.1_ASM26694v1_pr

GCF_000346485.2_ASM34648v2_protein.faa
>WP_017739643.1 30S ribosomal protein S4 [Scytonema hofmannii]
 MSRYRGPRLRVVRRLGDLPGLTRKSARRAYPPGQHGQNRKKRSEYAVRLEEKQKLRMNYGLTEKQLLRYVRKARRVTGSTGQVLLQLLEMRLDNTVFRLGMAPTIPAARQLVNHGHVTINGRVVNIASYQCRPGEVVAVRDREKSKELVTTNLQYPGLANLPSHLEFDKNKLVGKVNSVIEREWVALQVNELLVVEYYSRQA
GCF_001617355.1_Rhodanobacter_sp._FW104-T7_protein.faa
>WP_008435462.1 MULTISPECIES: 30S ribosomal protein S4 [Rhodanobacter]
 MARYRGATCKLARREGADLSLKSPARALDSKCKLENKPGQHGANKRMRMSDYAVQLREKQKVKRIYGVLERQFSNYYTKATTLKGNTGENLLRLLESRLDNVVYRMGFAVTRAQARQLVAHKAILVNGKKVNIPSYQVRPGDAIALTERARSQLRVQEAATVFDTMDLRPGWVEVDSKKFEGTFKSVPDRGDLPTDINEALIVELYSK
GCF_012999515.1_ASM1299951v1_protein.faa
>WP_111748154.1 MULTISPECIES: 30S ribosomal protein S4 [Salinisphaeraceae]
 MAKYTGPRCKLSRRTGTDLLLTSHGRSIESKCKFETPPGTQGSGRRQRLSDYATQLREKQKLRRMYGLMEKQFHKYYVKSSQRKGATGVLLLQALESRLDNVVYRMGFAATRAEARQLVNHKAVAVNDRLVNVPSYEVQPGDVVAIRNRAKNQNRIVEAMSVAAQLGFGDWLSVDDKKLQGTYTQVPDRDQILQDINENLVVELYSK
GCF_000423125.1_ASM42312v1_protei

In [139]:
checkGenome('GCF_008329785.1_ASM832978v1_protein.faa')

50S ribosomal protein L1 [
>WP_010888677.1 50S ribosomal protein L1 [Deinococcus radiodurans]
 MPKHGKRYRALEGKVDRNKQYSIDEAAALVKELATAKFDETVEVHFRLGIDPRKSDQNVRGTVALPHGTGRSVRVAVITKGENVQAAEAAGADVVGSDELIERIAGGFMDFDAVVATPDMMAQIGQKLARLLGPRGLLPNPKSGTVGADVAGMVRGLKAGRIEFRNDKTGVVHAPIGKASFESGNLSANYQALISALEGAKPGTAKGVFLRSAYLTTTMGPSIPLALGGAALA
1
50S ribosomal protein L2 [
>WP_010886959.1 50S ribosomal protein L2 [Deinococcus radiodurans]
 MAVKKYRPYTPSRRQMTTADFSGLTKKRPEKALTEALPKTGGRNNRGRITSRFIGGGHKRLYRIIDFKRRDKSGVNAKVAAIEYDPNRSARIALLHYADGEKRYILAPEGLTVGATVNAGPEAEPKLGNALPLRFVPVGAVVHALELVPGKGAQLARSAGTSVQVQGKESDYVIVRLPSGELRRVHSECYATIGAVGNAEHKNIVLGKAGRSRWLGRKPHQRGSAMNPVDHPHGGGEGRTGAGRVPVTPWGKPTKGLKTRRKRKTSDRFIVTRRK
1
50S ribosomal protein L3 [
>WP_010886956.1 50S ribosomal protein L3 [Deinococcus radiodurans]
 MKGILGTKIGMTQIWKNDRAIPVTVVLAGPCPIVQRKTAQTDGYEAVQIGYAPKAERKVNKPMQGHFAKAGVAPTRILREFRGFAPDGDSVNVDIFAEGEKIDATGTSKGKGTQGVMKRWNFAGGPASHGSKKWHRRPGSIGQRKTPGRVYKGKRMAGHMGMERVTVQNLEVVEIRAGENLILVKGAIPGANGGLVVLRSA

In [31]:
protein_name='50S ribosomal protein L1 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df.iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

50S ribosomal protein L1 [
GCF_000305785.2_ASM30578v2_protein.faa
>WP_015047194.1 50S ribosomal protein L1 [Simiduia agarivorans]
 MAKLTKRQRAIREKVDGAKLYSISEAVALLKEVSSVKFGETVDVAVNLGIDPRKSDQTVRGATTLPHGTGKSVRVAVFAQGANAEAAKAAGAEHVGMDELAAEIKGGLMDFDVVIAEPAAMRVVGALGQVLGPRGLMPNPKTGTVTPDVVTAVNNAKAGQVRYRAEKGGIVHGGIGKLSFEEKALKENLEALVGDLKRAKPASSKGVYLKRITLSTTMGPGLLIDQASLDI
GCF_000153125.2_ASM15312v2_protein.faa
>WP_008292631.1 50S ribosomal protein L1 [Congregibacter litoralis]
 MAKLSKRVKNAREKIDPTRAYSLDEAVSLLKEVSTVKFNETVEVAVKLGIDARKSDQAVRGATTLPHGTGSDVRVAVFAQGDAAEKATAAGADLVGMDDLAEQIKGGMMDFDVVVASPDAMRVVGQLGQVLGPRGLMPNPKTGTVTPDVETAVKNAKAGQMRYRTDKNGIVHGGIGKISFEADAIRGNLEAVLADLRKGKPAASKGVYMRKVTLSTTMGPGVTLDQTSLEA
GCF_000373945.1_ASM37394v1_protein.faa
>WP_020581777.1 50S ribosomal protein L1 [Endozoicomonas elysicola]
 MAKLSKRAKLIAEKIDSTKSYGFEEAATLLREISSVKFKETIEVSVNLGVDPRKSDQVVRGSTVLPNGTGKDVRVAVFTQGANAEAAKEAGAEIVGMEDLAERVKAGEMDFDVVIASPDAMRVVGQLGQILGPRGLMPNPKVGTVTPNVAEAVRNAKAGQVRFRTDKNGIIHAGVGKIDFEAAALKQNV

>WP_008224571.1 50S ribosomal protein L1 [Rheinheimera nanhaiensis]
 MAKLTKRAKLIRSKVDSTREYDINEAVALLKELTAGKFVESVDVAVNLGIDARKSDQNVRGATVLPHGTGRTVRVAVFTQGANAEAAKAAGADIVGMEDLAEQVKKGEMNFDVVIASPDAMRVVGMLGQILGPRGLMPNPKTGTVTPNVAEAVKNAKAGQVRYRNDKNGIVHSTIGKIDFDADKLKENLEALLVALKRAKPSVAKGIYIKKVTISSTHGAGLTLSQASLSAAV
GCF_009910945.1_ASM991094v1_protein.faa
>WP_159183777.1 50S ribosomal protein L1 [Francisella noatunensis]
 MAKISKRMKEISAKINAEKKYPVSEAFDILRDVSSVKFVESVDVSVALGVDPRKSDQVVRGASVLPNGTGKTVRVAVFAKGPAADTAKEAGADVVGMEDLADEVKKGNMDFDVVIASPDSMRVVGQLGQILGPKGLMPNPKVGTVTMDVAKAVRDAKAGQVRYRVDKAGIIHTTIGKVNFTSDALKQNLEQLLADLKKAKPSVSKGIYLKKVSVSSTMGPGISVDFSDLNI
GCF_000007765.2_ASM776v2_protein.faa
>NP_819271.1 50S ribosomal protein L1 [Coxiella burnetii RSA 493]
 MAAKLTKKRKTLAEKVQRDKTYPLSEAIKLIKECAKAKFNESIDVAINLGIDSRKSDQAIRGATVLPHGSGRTVKVAVFAQGDNVEKAKAAGADIVGLDDLAERIQGGDIDFDVVIATPETMRVVGKLGQVLGPRGLMPNPKVGTVTTDVASAVKNAKQGQVRYRTDKNGIIHCTIGKVNFEEKALEENFLALLNDIKKAKPSAAKGTYLKKLTLSSTMGPGIAIDRTTVGA
GCF_000239175.1_ASM2

>WP_011517782.1 MULTISPECIES: 50S ribosomal protein L1 [Cupriavidus]
 MAKVSKRVAANKAKIERTKFYPIDEALSLVKSCASAKFDESIDVAVQLGIDAKKSDQVVRGSVVLPAGTGKSVRVAVFAQGEKAEAAKAAGADVVGMEDLAEQVKAGNLDFDIVIASPDTMRIVGTLGQILGPRGLMPNPKVGTVTPDVAQAVKNAKAGQVQFRVDKAGIIHATIGRRSFEDTALKSNLSALLDALVKAKPATSKGVYLRKIAVSSTMGVGVRVEQTSLSA
GCF_000011545.1_ASM1154v1_protein.faa
>WP_004185135.1 MULTISPECIES: 50S ribosomal protein L1 [Burkholderia]
 MAKISKRRQAFAAKVDRQKLYPIDDALALVKECASAKFDESIDVAVQLGIDAKKSDQVVRGSVVLPAGTGKSVRVAVFAQGEKAEQARAAGAEVVGMEDLAEQIKAGQMDFDIVIASPDTMRIVGTLGQILGPRGLMPNPKVGTVTPDVATAVKNAKAGQVQFRVDKAGIIHATIGRASFEPTALRTNLSALIEALQKAKPATSKGVYLRKIALSSTMGVGVRVDQGSLAAQ
GCF_000250875.1_ASM25087v1_protein.faa
>WP_008540549.1 50S ribosomal protein L1 [Sutterella parvirubra]
 MAKLTKRMQALQGKVEAGKVYPALDALNIVKSAATAKFDESIDVAIQLGIDPRKSDQAVRGAVVMPEGTGKTVRVAVFTQGANADAAREAGADIVGFDDLAAKVKAGELDFDVVIASPDAMRVVGQLGQILGPRGLMPNPKVGTVTPNVAQAVKNAKAGQVQFRADKAGIIHAPIGRASFEAERLQKNLAALIEQLNKLKPASAKGQYLRKVAVSSTMGLGVRVDVGSVNNA
GCF_000318015.1_AS

GCF_000661935.1_ASM66193v1_protein.faa
>WP_039147443.1 50S ribosomal protein L1 [Bradyrhizobium japonicum]
 MAIGKRLNKAREGVDREKLYPLTDAIKMVKERAKAKFDETIEVAINLGVDPRHADQMVRGVVTLPNGTGRTLRVGVFARGAKADEAKAAGADVVGAEDLVEKVQNGSIDFDRCIATPDMMPLVGRLGKVLGPRGLMPNPKIGTVTMDVTGAVKGAKGGSVEFRVEKAGILQAGVGKASFSEEKLVENIKALADAVSKAKPAGSKGTYIQRVAVSSTMGPGVKVEPGTILG
GCF_000192745.1_ASM19274v1_protein.faa
>WP_013652472.1 50S ribosomal protein L1 [Polymorphum gilvum]
 MAKIGKRIRAARDGIDRNKTYPLSEALALIKERAKVKFDETVEVAINLGVDPRHADQMVRGVCILPNGTGKTVRVAVFARGDKADEAKAAGADIVGAEELVNEVQGGKIDFDRCIATPDMMPLVGRLGKVLGPRGLMPNPKVGTVTPDVASAVRDAKGGAVQFRVEKAGIVHAGVGKVSFSQDALVENIRALLDAVQKAKPSGSKGTYLKRVALSSTMGPGVKVDPSTIAAG
GCF_000421645.1_ASM42164v1_protein.faa
>WP_024352398.1 50S ribosomal protein L1 [Aurantimonas coralicida]
 MAKSGKRIRSAREGIERTKLYGLDEAVQMLKDRTKTKFDETIEVAMNLGVDPRHADQMVRGVVNLPNGTGRTVRVAVFARGDKAEEAKAAGADIVGAEDLVEQVQGGTIDFDRAIATPDMMALVGRLGKVLGPRGLMPNPRVGTVTPDVAAAVKASKGGAVEFRVEKAGIVHAGVGKASFDSKAIVENIRAFADAVQKAKPSGAKGNYVQRIAVTSTMG

>WP_106459294.1 50S ribosomal protein L1 [Aphanothece hegewaldii]
 MTKKVSRRLRDARAKVDDKAYEALEALQLLKETATAKFDETAEAHIRLGIDPKYTDQQLRTTVTFPKGTGQTVRVAVITRGEKESEAKAAGADIVGSEDLINQIQGGMLDFEVLIATPDMMPQVAKLGRLLGPRGLMPSPKGGTVTTDLAGAIEEFKAGKQEFRADRTGIVHVMFGKASFTADDLLVNLKALQETVDRNRPTGAKGRYWRSVYVSASMGPSIEVDISSLRDLKLAEA
GCF_000210375.1_ASM21037v1_protein.faa
>WP_006619242.1 50S ribosomal protein L1 [Arthrospira platensis]
 MTKKVSRRLQELQQKVEDRLYEPLEGLTLLKETATAKFVESVEAHIRLGIDPKYTDQQLRTTVTLPKGTGQEVRVAVLAKGDKVAEAQSAGADLVGSDDLIAEIQKGMLDFDKLIATPDMMPQVAKLGRLLGPKGLMPSPKGGTVTMDLTQAIAEFKAGKLEFRSDRTGIVHVMFGKVSFSAEDLLVNLSALQECIDRNRPSGAKGRYWRSVYVCSTMGPAIQLDINALRDLKIAEAA
GCF_000281175.1_ASM28117v1_protein.faa
>WP_014432762.1 50S ribosomal protein L1 [Caldilinea aerophila]
 MAKHSKRYLALRQKVDRTKAYPPKEAIELVKEMANAKFDETVEVHMRMGVDPRHADQQVRGVVTLPNGTGRTVRILVFANPEGQALAREAGADYVGAEDLADQIMKGWTDFDVVLATPDMMRVVGRLGKVLGPRGLMPSPKTGTIVPAEDLPRVINEARLGRVEFRVDKTSNIHVPIGKSSFPTEKLLQNLGALMEAVIAAKPPTLKGNYIQRITLTSTMSPGVKADLASTMALKADAA
GCF_003

...

GCF_000143965.1_ASM14396v1_protein.faa
>WP_013259739.1 50S ribosomal protein L1 [Desulfarculus baarsii]
 MAKRGKQLVAARGKIDRSKRYGFEEGIQAALEAKFAKFDESVDVAVRLGVDPRHADQMIRSTVVLPHGTGKTVRVAVFAKGEKAQEARDAGADEVGAEDLLEKIQGGWLEFDRCVATPDMMAVVGKLGRVLGPRGLMPNTKTGTVTFDLAKAVKEIKAGKIEFRVEKGGIIHAPIGKASFGVEKLKENMEALLETLIRLKPTAAKGTYMKGIAVSTTMGPGVRLDVAEVKAISAKG
GCF_001184205.1_ASM118420v1_protein.faa
>WP_049677279.1 50S ribosomal protein L1 [Desulfocarbo indianensis]
 MAKHGKNYKAAADKVDRQQRYTFEEGVAVAVSTGCAKFDETVDVAVRLGVDPRHADQMVRGTVALPHGTGKEVRVAVFAKGEKEKEAQDAGADFVGAEDLAEKIQGGWTDFDRAIATPDMMAVVGKLGRILGPRGLMPNTKSGTVTFELAKAVKEIKGGKIEFRVDKGGVVHAPIGKRSFGAEKIRENLNSLVETLLRLKPATAKGTYIKGISIATTMGPGVKLDPAEVKALVK
GCF_004319545.1_ASM431954v1_protein.faa
>WP_111956843.1 50S ribosomal protein L1 [Desulfobacter hydrogenophilus]
 MPKRSKKHNEALSNVDRMVQYGPKDALEIAVSSSYAKFDETVDVAVRLGVDPRHADQMVRGTVVLPNGLGKEVKVLVFAKGEKEQEALDAGADFIATDEIVEKIKDGWFGFDKAIATPDMMGTVGKLGRVLGPRGLMPNAKTGTVTFELAKAINEVKAGKIDFRVEKAGIVHVPVGKISFGAEKLLENVTVFLDKIVSLK

>WP_013322792.1 50S ribosomal protein L1 [Gloeothece verrucosa]
 MPKKISRRLKEALAKVDENKAYEPLEALNLLKETATAKFDETAEAHVRLGIDPKYTDQQLRTTVTFPKGTGQTVRVAVIARGEKVQEASNGGADIVGSEELIDEIQNGMMDFEVLIATPDMMPKVAKLGRLLGPRGLMPSPKGGTVTTDLAAAIQEFKAGKQEFRADRTGIVHVMFGKASFSAEDLLVNLKALQETIDRNRPSGAKGRYWRTIYVSASMGPSIQVDVNSLRDLKAEA
GCF_000258405.1_ASM25840v1_protein_EDIT_ADD_GENBANK_AFH48648.1.faa
>WP_014559769.1 50S ribosomal protein L1 [Ignavibacterium album]
 MQLTKRNKKILEKVDTKKEYLIEEAIKTLKELSSVKFVESLDVAIRLGVDPRHADQMVRGTVSLPHGTGKEVKVLVIAKGPKIQEALDAGADYAGFEEYLEKIKGGWADVDVIIATPDTMAELGKLGKVLGPKGLMPNPKSGTVTMDVAKAVKEVKAGKIEFRVEKTGIVHTSVGKLNFEVDKLVENTKAFLNTIIKLKPPTAKGQYVKSLYLSTTMGPGLKISKEEIAI
GCF_000178855.1_ASM17885v1_protein_EDIT_ADD_GENBANK_EFH90870.1.faa
>WP_007913380.1 50S ribosomal protein L1 [Ktedonobacter racemifer]
 MAIKKHGKKYLEVSKLLEVEKLYEPKEAVELLKKLNYVKFDPTVEVHMKLGVDPRHADQMVRGVAMLPAGTGKEVKVLVFAQGEKVSEAETAGADFVGLDDLIKQIEGDWLGFDVAIATPDVMGKVGRLGKKLGPRGLMPSPKAGTITFDLAKTIREVKAGRVEFKVDKTALLHIPAGKLSFSEDSLMSNITSVIDAV

>NP_628810.1 50S ribosomal protein L1 [Streptomyces coelicolor A3(2)]
 MSKRSKSLRAADAKIDRDKLYAPLEAVRLAKETSTSKFDGTVEVAFRLGVDPRKADQMVRGTVNLPHGTGKTARVLVFATGDRAEAATAAGADIVGSDELIDEVSKGRLDFDAVVATPDLMGKVGRLGRVLGPRGLMPNPKTGTVTPDVAKAVNDIKGGKIEFRVDKHSNLHFIIGKTSFDDTKLVENYGAALEEILRLKPSAAKGRYIKKAALSTTMGPGIPLDSNRTRNLLVEEDPAAV
GCF_004769835.1_ASM476983v1_protein.faa
>WP_012388948.1 MULTISPECIES: 50S ribosomal protein L1 [Leptospira]
 MKRGKKYIQLKEKVDRTKAYTLGEAVGLAKATSYSKFDGTLEISTKINYKSLQNVRGTISLPHGTGKTIKVLVFCKGDKQNEAREAGADFVGDMDLIEKVSGGWTDFDACVATPDMMKEVGKLGPVLGRKGLMPKPKAGTVTTDVSKAVKELKAGRIEYRPDKGGVVHLGVGKCSFSDDKLSDNINAVVAALMKDKPSDAKGDYLKSFSVAATMGIGVKVDVKELVNANI
GCF_000214355.1_ASM21435v1_protein.faa
>WP_015710711.1 50S ribosomal protein L1 [Treponema azotonutricium]
 MKHGKKYRESIKKYNPESSYELKEALALVKTMAFAKFDETVDLSVKVSLKKSQSVRDTVVLPNQFRGEKKVLVFCKPEKEKEAQEAGAAFVGADDLIEKVKGGWTDFDIAVATPDMMKDVGKLGMVLGRKGLMPNPKTGTVTFDLKGALAELRKGRTEFRADKTGVVHLAVGKVSMEPEKVAENVQIVVSEINRKRPADAKGDFLQSISVASTMGPGVWVTAAKEGE
GCF_0002974

>WP_045116851.1 50S ribosomal protein L1 [Plesiocystis pacifica]
 MGKKYKEARSKIDRDKRYGLVEACELVKVVAPAKFDETVECAARLGVNPRHADQMIRSSVSLPHGTGKSVRVLVFAKGDRVNQAKDAGADFVGSDDIVAKIQEGWLDFDVAIATPDMMGLVGRLGRILGPRNLMPNPKVGTVTMDVKKAVTERKAGRIEFRTERTGIIHAPIGKVSFEAVKLQENLETFLRTLQRLKPATAKGKYFKSVSLSSTMGPGIRLDTNEVQSMVTAE
GCF_001273775.1_ASM127377v1_protein_EDIT_ADD_L10.faa
>WP_053378937.1 50S ribosomal protein L1 [Nitrospira moscoviensis]
 MGKKMKAALEKVEPRAYALREAVDVVKKAAYAKFDESVDLALRLGIDPKRSDQMVRGTTSLPHGTGKQVRVVVFAKGEKEQEARQAGADFVGADDLMEKIKGGWMDFDYAIATPDLMASVGKLGKQLGPRGLMPNPKTGTVTFEVGKAVGEIKKGRVEFKVEKAGIVQVPVGKVSFDVNKLYDNAAAIIESVIKAKPASCKGRYLKSATISSTMGPGVQLDTVALTKQWS
GCF_000025705.1_ASM2570v1_protein.faa
>WP_013028895.1 50S ribosomal protein L1 [Sideroxydans lithotrophicus]
 MAQVSKRYKALAAKVDRNKLYALDEALKLVKETAKAKFDESIDVAVNLGIDAKKSDQLVRGSVVLPKGTGKTMRVAVFAQGAKAEEAKAAGADIVGFDDLAESIKAGKMDFDVVIASPDAMRIVGQLGQILGPRGLMPNPKVGTVTADVAGAVKNAKAGQVQYRTDKNGIVQCTIGRASFAPEALRENMLALIDALNKAKPATSKGVYLKKVAISSTMGVGIRVEQASLVA
GCF_00

GCF_000702665.1_ASM70266v1_protein.faa
>WP_029461033.1 50S ribosomal protein L1 [Desulfovibrio alcoholivorans]
 MAKHGKKYREAVKDIDLQAKFDVNEAMALTVKTGVAKFDETVDVALCLGVNPKYSDQMVRGAVSLPHGLGKTVRVLAFCKGDKEAEAREAGADFAGGDDLIEKIKGGFLDFDKAVATPDMMATVGKIGKILGPRGLMPNAKTGTVTFDIGKAVQELKAGKVEFKVDKAGVLHVPLGKRSFGPEKLLDNFRALIDTVMRLKPSAAKGTYLQAMALATTMGPGIKVDTQSVRKLLEG
GCF_000010665.1_ASM1066v1_protein.faa
>WP_015861767.1 50S ribosomal protein L1 [Desulfovibrio magneticus]
 MAKHGKNYREAIKDIDLTAKYEVSEAMALTVQTGKAKFDETVDVALNLGVNPKYSDQMVRGAVSLPHGLGKTIRVAAFCKGDKETEAREAGADFVGGDDLIEKVKNGFLDFDSAVATPDMMASVGKIGKILGPRGLMPNAKTGTVSFDIGKAVSELKAGKVEFKVDKAGVLHVPLGKRSFGPEKLLDNFRAVIDAVMRLKPSAAKGTYLQAMALATTMGPGIKVDTQSVRKLLEG
GCF_900108475.1_IMG-taxon_2597490343_annotated_assembly_protein.faa
>WP_074767920.1 50S ribosomal protein L1 [Magnetospirillum fulvum]
 MAREGKRLKAAYAGINRDKFYTLAEAVAVIKSSATSKFDETVEIALNLGVDPRHADQMVRGVIELPHGTGKTVRVAVFAKGDKAEEAKKAGADIVGAEDLFEAVNGGTMDFDRCIATPDMMGLVGRLGKVLGPRGLMPNPKLGTVTTNVVEAVRAAKAGQVQFRVEKAGIIHA

In [30]:
### Checking Routine

In [44]:
#Look for all Large subunits in all Taxa, print those that don't have 15.
#There's got to be a better way to do this!!!
num_complete = 0    
for protein_name in large_subunits+small_subunits:
    num_complete = 0
    print(protein_name)
    for index,row in taxa_df.iterrows():
        #print(index) #Turn this on to get specific.
        index_complete = 0
        for header, seq in fasta_iter(index):
            index_complete = index_complete+1
            if header.find(protein_name) != -1:
                #print(header,seq) #Turn this on to get specific.
                num_complete = num_complete+1
            else:
                #print('ERROR!',index,protein_name)
                continue
    print(num_complete)

50S ribosomal protein L1 [
306
50S ribosomal protein L2 [
306
50S ribosomal protein L3 [
306
50S ribosomal protein L4 [
306
50S ribosomal protein L5 [
306
50S ribosomal protein L6 [
306
50S ribosomal protein L10 [
306
50S ribosomal protein L13 [
306
50S ribosomal protein L14 [
306
50S ribosomal protein L15 [
306
50S ribosomal protein L18 [
306
50S ribosomal protein L22 [
306
50S ribosomal protein L23 [
306
50S ribosomal protein L24 [
306
50S ribosomal protein L29 [
306
30S ribosomal protein S2 [
306
30S ribosomal protein S3 [
306
30S ribosomal protein S4 [
306
30S ribosomal protein S5 [
306
30S ribosomal protein S7 [
306
30S ribosomal protein S8 [
306
30S ribosomal protein S9 [
306
30S ribosomal protein S10 [
306
30S ribosomal protein S11 [
306
30S ribosomal protein S12 [
306
30S ribosomal protein S13 [
306
30S ribosomal protein S14 [
306
30S ribosomal protein S15 [
306
30S ribosomal protein S17 [
306
30S ribosomal protein S19 [
306


In [64]:
#Look for all Large subunits in all Taxa, print those that don't have 15.
#There's got to be a better way to do this!!!
num_complete = 0    
for protein_name in large_subunits+small_subunits:
    num_complete = 0
    print(protein_name)
    for index,row in taxa_df[227:].iterrows():
        #print(index) #Turn this on to get specific.
        index_complete = 0
        for header, seq in fasta_iter(index):
            index_complete = index_complete+1
            if header.find(protein_name) != -1:
                #print(header,seq) #Turn this on to get specific.
                num_complete = num_complete+1
            else:
                #print('ERROR!',index,protein_name)
                continue
    print(num_complete)

50S ribosomal protein L1 [
80
50S ribosomal protein L2 [
80
50S ribosomal protein L3 [
80
50S ribosomal protein L4 [
80
50S ribosomal protein L5 [
81
50S ribosomal protein L6 [
80
50S ribosomal protein L10 [
78
50S ribosomal protein L13 [
80
50S ribosomal protein L14 [
80
50S ribosomal protein L15 [
80
50S ribosomal protein L18 [
79
50S ribosomal protein L22 [
79
50S ribosomal protein L23 [
79
50S ribosomal protein L24 [
81
50S ribosomal protein L29 [
79
30S ribosomal protein S2 [
78
30S ribosomal protein S3 [
79
30S ribosomal protein S4 [
85
30S ribosomal protein S5 [
81
30S ribosomal protein S7 [
80
30S ribosomal protein S8 [
80
30S ribosomal protein S9 [
80
30S ribosomal protein S10 [
80
30S ribosomal protein S11 [
80
30S ribosomal protein S12 [
80
30S ribosomal protein S13 [
80
30S ribosomal protein S14 [
85
30S ribosomal protein S15 [
79
30S ribosomal protein S17 [
80
30S ribosomal protein S19 [
80


In [143]:
!pwd -P

/nobackup1c/users/payette/ABGDtree5/sequences


In [45]:
taxa_df.loc[taxa_df.Filename=='GCF_000317635.1_ASM31763v1_protein.faa']

,Index,Name,URL,Filename,FullURL,Accession,TaxID,Comments,TaxonomyNotes
Filename,,,,,,,,,
GCF_000317635.1_ASM31763v1_protein.faa,173,Halothece sp. PCC 7418,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000317635.1_ASM31763v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000317635.1,65093,Genera added from Greg,NaN


In [46]:
#All in one -- one shot, iterate over genome list, grab fasta, rewrite, print to screen
#Manually build files.
#Look for all Large subunits in all Taxa, print those that don't have 15.
#There's got to be a better way to do this!!!
num_complete = 0    
for protein_name in large_subunits+small_subunits:
    num_complete = 0
    print(protein_name,'### BREAK ###')
    for index,row in taxa_df.iterrows():  #[taxa_df.Name=='Desulfurella acetivorans A63']
        print(index) #Turn this on to get specific.
        index_complete = 0
        for header, seq in fasta_iter(index):
            index_complete = index_complete+1
            if header.find(protein_name) != -1:
                
                new_header = taxa_df.loc[index][1]
                new_header =  new_header.replace(' ', '_')
                
                print('>'+new_header+'\n'+seq) #Turn this on to get specific.
                num_complete = num_complete+1
            else:
                continue
                print('ERROR!',index,protein_name)
    #print(num_complete)

50S ribosomal protein L1 [ ### BREAK ###
GCF_000305785.2_ASM30578v2_protein.faa
>Simiduia_agarivorans_SA1_=_DSM_21679
MAKLTKRQRAIREKVDGAKLYSISEAVALLKEVSSVKFGETVDVAVNLGIDPRKSDQTVRGATTLPHGTGKSVRVAVFAQGANAEAAKAAGAEHVGMDELAAEIKGGLMDFDVVIAEPAAMRVVGALGQVLGPRGLMPNPKTGTVTPDVVTAVNNAKAGQVRYRAEKGGIVHGGIGKLSFEEKALKENLEALVGDLKRAKPASSKGVYLKRITLSTTMGPGLLIDQASLDI
GCF_000153125.2_ASM15312v2_protein.faa
>Congregibacter_litoralis_KT71
MAKLSKRVKNAREKIDPTRAYSLDEAVSLLKEVSTVKFNETVEVAVKLGIDARKSDQAVRGATTLPHGTGSDVRVAVFAQGDAAEKATAAGADLVGMDDLAEQIKGGMMDFDVVVASPDAMRVVGQLGQVLGPRGLMPNPKTGTVTPDVETAVKNAKAGQMRYRTDKNGIVHGGIGKISFEADAIRGNLEAVLADLRKGKPAASKGVYMRKVTLSTTMGPGVTLDQTSLEA
GCF_000373945.1_ASM37394v1_protein.faa
>Endozoicomonas_elysicola_DSM_22380
MAKLSKRAKLIAEKIDSTKSYGFEEAATLLREISSVKFKETIEVSVNLGVDPRKSDQVVRGSTVLPNGTGKDVRVAVFTQGANAEAAKEAGAEIVGMEDLAERVKAGEMDFDVVIASPDAMRVVGQLGQILGPRGLMPNPKVGTVTPNVAEAVRNAKAGQVRFRTDKNGIIHAGVGKIDFEAAALKQNVEALLVDLKKLKPSSSKGVYMKKVSLSSTMGPGLTIDMSSLTA
GCF_000647595.2_PseAer148_2.0_protein.faa

>Thioalkalimicrobium_aerophilum_AL3
MAKLTKKQKAMAERINKDSAYEVIAALDLVKELATAKFDETVDVAVKLGVDPRKSDQVVRGATVLPNGLGKTIRVAVFTGEANQAAAKAAGADFVGMDELAAEIKKGMMDFDVVIASPDAMRVVGALGTVLGPRGLMPNPKTGTVTPDVVTAINNAKAGQVRYRVDKAGIVHAPIGKASFDSQKLKENLAALIEDLNKAKPSAAKGVYMKKVSLSSTMGPAVTVEQSSI
GCF_000087965.2_ASM8796v1_protein.faa
>Xanthomonas_albilineans_GPE_PC73
MAQTKRQKAIRAAVQPGKAYSIDEALKIIKTTSKAKFVEAVDVAVRLGVDAKKSDQQVRGSTVLPAGTGKSVRVAVFAPAGAKADEALAAGAEAVGMDDLAEKMQGGDLNYDVVIATPDAMRVVGKLGTLLGPRGLMPNPKVGTVSANPAEAVKNAKSGQVRYRTDKAGIIHCTIGKASFEDDALKSNLHALLLDLVKAKPATSKGNYLQKISVSSTMGPGVTIDQASLSLK
GCF_000007245.1_ASM724v1_protein.faa
>Xylella_fastidiosa_Temecula1
MVQTKRQKAIDIAVVPGKAYGIDEAIKILKTATKAKFIESVDVAIRLGVDVKKSDQQVRGSTLLPAGTGRDVRVAVFVPSGAKAEDALAAGADAVGMDDLAEKMQAGDLNYDVVIATPDAMRVVGKLGTLLGPRGLMPNPKVGTVSQNPGEAVKNAKSGQVRYRADKAGIIHCVIGKVNFDDEALKLNLQALLVDLIKIKPTASKGTYLQKVSLSSTMGPGVMIDQSTLSLK
GCF_000024765.1_ASM2476v1_protein.faa
>Halothiobacillus_neapolitanus_c2
MAKLTKRQKAIAAKVNANQPYAAADAFNLLKEVSTVKFVEAVDVSVHLGVDPRRSDQ

GCF_000963925.1_ASM96392v1_protein.faa
>Acetobacter_cibinongensis
MATKKRLAAAQAKVNSAQAYGLDEAIALVKGNATAKFDETIEISLNLGIDPRHADQMVRGFVSLPNGTGKTLRVGVFARGAKAEEAQAAGADVVGAEDLMEKVQAGEINFDRCIATPDMMALVGRLGKILGPRGLMPNPKLGTVTMDVKGAIEAAKSGQVEYRAEKAGIVHAGVGKASFDAAKLAENVRAFVDAVQKARPTGAKGTYMKKAGLSSTMGPGVQLDLSAFAG
GCF_000013085.1_ASM1308v1_protein.faa
>Rhodospirillum_rubrum_ATCC_11170
MAKVGKRLKAAHEGIDREAFYDLPQAVHLIRERATAKFDESIEVAMNLGVDPRHADQMVRGVVELPHGTGKTMRVAVFAKGAKAEEAKAAGADLVGADELAADIQNGTIDFDRCIATPDMMGVVGRLGKVLGPRGLMPNPKLGTVTMDVATAVKAAKAGQVQFRVEKSGIVHAGVGRASFTEEKLVDNVRAFVDAINKAKPTGAKGTYLKKVSLSSTMGPGLKVNLGTISGAA
GCF_000767795.1_Ilimosus_MP06_1.0_protein.faa
>Inquilinus_limosus_MP06
MAKVSKRIRAATKGLNPEATYELAQAVKMLKASATAKFDETVEIAMNLGVDPRHADQMVRGVVQLPNGTGKTVRVLVFAKGDKAEQALAAGADIVGADDLAKEIQDGRSDFDRCIATPDMMGLVGRLGKVLGPRGLMPNPKLGTVTPDVAGAVKAAKGGAVEFRAEKAGIVHAGVGKASFTEEALVGNIRAFVEAIQRAKPSGAKGTYLKKVTVTSTMGPGYKLDLQKLHEGAGA
GCF_000013005.1_ASM1300v1_protein.faa
>Erythrobacter_litoralis_HTCC2594
MAKQTKKQQEVAKLDPEK

GCF_000199675.1_ASM19967v1_protein.faa
>Anaerolinea_thermophila_UNI-1
MAKHGKKYLQALAKIDPNRVYEPKQALELIKEISYANFDATVEVHMRTGLDPRQADQQVRDVVVLPHGLGKTVRVLVFAQGDGATLAQQAGADYVADTDEWIQKIQGGWTDFDVAIATPDMMGKVGRLGRILGPRGLMPNPKAGTVVPAEDLPRVIKESKAGRVEFRLDKSANLHVPIGKVSFDVEKLYENLSALMEAIRKARPAAAKGVYIRRVTITPTMGPGVRIDPNVALSLTVK
GCF_003007785.1_ASM300778v1_protein.faa
>Aphanothece_hegewaldii_CCALA_016
MTKKVSRRLRDARAKVDDKAYEALEALQLLKETATAKFDETAEAHIRLGIDPKYTDQQLRTTVTFPKGTGQTVRVAVITRGEKESEAKAAGADIVGSEDLINQIQGGMLDFEVLIATPDMMPQVAKLGRLLGPRGLMPSPKGGTVTTDLAGAIEEFKAGKQEFRADRTGIVHVMFGKASFTADDLLVNLKALQETVDRNRPTGAKGRYWRSVYVSASMGPSIEVDISSLRDLKLAEA
GCF_000210375.1_ASM21037v1_protein.faa
>Arthrospira_platensis_NIES-39
MTKKVSRRLQELQQKVEDRLYEPLEGLTLLKETATAKFVESVEAHIRLGIDPKYTDQQLRTTVTLPKGTGQEVRVAVLAKGDKVAEAQSAGADLVGSDDLIAEIQKGMLDFDKLIATPDMMPQVAKLGRLLGPKGLMPSPKGGTVTMDLTQAIAEFKAGKLEFRSDRTGIVHVMFGKVSFSAEDLLVNLSALQECIDRNRPSGAKGRYWRSVYVCSTMGPAIQLDINALRDLKIAEAA
GCF_000281175.1_ASM28117v1_protein.faa
>Caldilinea_aerophila_DSM_14535

GCF_000183135.1_ASM18313v1_protein.faa
>Paludibacter_propionicigenes_WB4
MSKLTKNQRLALEKIEAGKTYSLKEAAALVKEITNTKFDASVDIDVRLGVDPRKANQMVRGVVSLPNGTGKQVKVLALCTPDQEAAAKEAGADFVGLDEYIEKIKGGWTDVDVIITQPAIMGKLGALGRVLGPRGLMPNPKSGTVTNEVGKAVKEVKQGKIDFKVDKYGIVHTSVGKVSFTAEQIVENAKEFVSTLMKLKPTAAKGTYVKSIYLSSTMSSGIKIEPKSIEE
GCF_000812965.1_ASM81296v1_protein.faa
>Pedobacter_kyungheensis
MAKLTKNQKKAHAKIEAGKAYTLKDAAALVKEITTTKFDASVDIDVSLGVDPRKANQMVRGIATLPHGTGKTVRVLALVTPDKEEEAKAAGADFVGLDEYVAKIEGGWTDVDIIITTPACMAKVGKLGRVLGPRNLMPNPKSGTVTNEVGKAVTEVKAGKIDFKVDKTGIIHASIGKVSFPADKIYENALEIIQVISKLKPSAAKGTYFKSIHVSSTMSPGIAIETKSVAGI
GCF_000723505.1_Mucivorans_M3_Nelson-Bomar-Fixed_protein.faa
>Mucinivorans_hirudinis
MKLTKNRKAVLAKLEPNKVYKLADAAALLKEITFTKFDASVDIDVRLGVDPRKANQMVRGVVTLPHGTGKDVRVLVLCTPDKEAEAQAAGADHVGLDEFVEKIKGGWTDVDVIICTPNVMGKVGALGRILGPRGLMPNPKTGTVTMDVAQAVKEVKAGKIDFKVDKFGIVHTSVGKISFDADKIVDNANEFLKTIIKLKPVAAKGTYVKSIYVSTTMSPGLQIDTKSVE
GCF_000736515.1_ASM73651v1_protein.faa
>Schleiferia_thermophila_str._Yellowstone
MA

...

GCF_000441555.1_ASM44155v1_protein.faa
>Profftella_armatura
MARSLKKGPFCDVHLIKKVEKAQNAKDKKPIKTWSRRSTIMPDFIGLTISVHNGKEHIPIYISENMVGHKLGEFALTRTFKCHSGDKKAKK
GCF_000158475.2_Oxal_for_HOxBLS_2_V2_protein.faa
>Oxalobacter_formigenes_HOxBLS
MTRSLKKGPFCDAHLIAKVEAAQASKDKRPIKTWSRRSTIMPDFIGLTIAVHNGKQHVPVYISENMVGHKLGEFALTRTFKGHAADKKAKK
GCF_000196015.1_ASM19601v1_protein.faa
>Cupriavidus_metallidurans_CH34
MTRSAKKGPFCDGHLLKKVEAAVGGKDKKPIKTWSRRSTILPDFIGLTIAVHNGRQHVPVYVTENMVGHKLGEFAITRTFKGHAADKKAKR
GCF_000011545.1_ASM1154v1_protein.faa
>Burkholderia_pseudomallei_K96243
MARSVKKGPFCDAHLLKKVEAAAASRDKKPIKTWSRRSTILPDFIGLTIAVHNGRQHVPVYISENMVGHKLGEFALTRTFKGHAADKKAKK
GCF_000250875.1_ASM25087v1_protein.faa
>Sutterella_parvirubra_YIT_11816
MSRSLKKGPFVDGHLMKKVEAAQASRDKRPLKTWSRRSTILPEFIGLTIAVHNGRQHVPVYINENMVGHKLGEFANTRTFKGHAAGDKKVGK
GCF_000318015.1_ASM31801v1_protein.faa
>Bordetella_bronchiseptica_253
MSRSIKKGPFVDAHLIKKVDAAVAGKDKKPIKTWSRRSTVLPEFIGLTIAVHNGRQHVPVYINENMVGHKLGEFALTRTFKGHAADKKSKR
GCF_000340905.1_ASM340

GCF_000210375.1_ASM21037v1_protein.faa
>Arthrospira_platensis_NIES-39
MSRSLKKGPFVADHLLVKIEKLNADNRKEVIKTWSRSSTIVPQMVGHTIAVHNGRQHVPVFVNEQMVGHKLGEFAPTRTFRGHAKSDKKARR
GCF_000281175.1_ASM28117v1_protein.faa
>Caldilinea_aerophila_DSM_14535_=_NBRC_104270
MSRSLKKGPFVSPKLLKRIEEMNRKGERKVIKTWSRASTIFPQFVGHTIAVHDGRRHVPVYISENMVGHKLGEFAPTRFFRGHVKSEKSSKTTGRR
GCF_003003845.1_ASM300384v1_protein.faa
>Chamaesiphon_polymorphus_CCALA_037
MGRSLKKGPFVADHLMTKVEKMNASGRKDVIKTWSRASTILPQMVGHTIAVHNGRQHVPVFVSDQMVGHKLGEFAPTRNYRSHSKGDKKSGR
GCF_000018865.1_ASM1886v1_protein.faa
>Chloroflexus_aurantiacus_J-10-fl
MSRSSKKGPYVDIKLLDKIEAMNKANEKRVIRTWSRDSTIFPQMIGHTIAVHDGRRHVPVYITENMVGHKLGEFAPTRFFRGHGGKKADKRGKVK
GCF_003003885.1_ASM300388v1_protein.faa
>Chlorogloea_sp._CCALA_695
MGRSLKKGPFVADSLLTKVEKLNAKGEKQVIKTWSRASTILPQMVGHTIAVHNGRQHIPVFLSEQMVGHKLGEFAPTRTFRGHSKSDKKAGR
GCF_000317125.1_ASM31712v1_protein.faa
>Chroococcidiopsis_thermalis_PCC_7203
MGRSLKKGPFIADSLLSKIEKLNSRGEKQVIKTWSRASTILPQMVGHTIAVHNGRQHVPIYVSEQMVGHKLGEFAPTRTFR

GCF_000478825.2_ASM47882v2_protein.faa
>Synechocystis_sp._PCC_6714
MGRSLKKGPFVAASLLRKIDKLNDKGDKQVVKTWSRASTILPQMVGHTIAVHNGRQHVPVFVSEQMVGHKLGEFAPTRTFRSHSKSDKKARK
GCF_000478195.2_ASM47819v2_protein.faa
>Lyngbya_aestuarii_BL_J
MSRSLKKGPFVADHLLKKIAALDASGKKGKEVIKTWSRASTILPLMVGHTIAVHNGRQHVPVFVTEQMVGHKLGEFAPTRTYRSHAGGDKKARR
GCF_000317045.1_ASM31704v1_protein.faa
>Geitlerinema_sp._PCC_7407
MSRSLKKGPFVADHLMRKIEAMNAKGDKQVIKTWSRASTIIPNMIGHTIAVHNGRQHVPVYVTEQMVGHKLGEFAPTRTFRGHAKSDKKARR
GCF_000317495.1_ASM31749v1_protein.faa
>Crinalium_epipsammum_PCC_9333
MGRSLKKGPFIADHLLTKIEKLNAKGEKQVIKTWSRASTVLPQMVGHTIAVHNGRQHVPVFLNEQMVGHKLGEFAPTRTFRGHSKSDKKAGR
GCF_000317635.1_ASM31763v1_protein.faa
>Halothece_sp._PCC_7418
MGRSLKKGPYVDDKLLKKIEQLNAADKKEVIKTWKRASTIIPQMVGHTIAVYNGRQHVPVFVSDQMVGHKLGEFAPTRTFKGHAKSDKKARR
GCF_003258315.1_ASM325831v1_protein.faa
>Bradymonas_sediminis
MPRSIKKGPFVDAGLRKKVTRAKDAGDRRALKTWSRRSMVVPEFLGMTFAVHNGKDFVPVFVTENMIGHKLGEFAPTRTYYGHVVETKGKLRGR
GCF_003260125.1_ASM326012v1_protein.faa
>Lujinx

GCF_000091545.1_ASM9154v1_protein.faa
>Thermus_thermophilus
MPRSLKKGVFVDDHLLEKVLELNAKGEKRLIKTWSRRSTIVPEMVGHTIAVYNGKQHVPVYITENMVGHKLGEFAPTRTYRGHGKEAKATKKK
GCF_000376665.1_ASM37666v1_protein.faa
>Meiothermus_ruber
MPRSLKKGVFVESHLLEKVEAMNAKGEKRVIKTWSRRSTIVPEMIGHTLAVYNGKQHVPVYITEQMVGHKLGEFSPTRTYRGHSKEAKTAKK
GCF_000195955.2_ASM19595v2_protein_EDIT_RM_S14.faa
>Mycobacterium_tuberculosis
MPRSLKKGPFVDEHLLKKVDVQNEKNTKQVIKTWSRRSTIIPDFIGHTFAVHDGRKHVPVFVTESMVGHKLGEFAPTRTFKGHIKDDRKSKRR
GCF_001457455.1_NCTC11397_protein_RM_L5_L24.faa
>Corynebacterium_diphtheriae
MPRSLKKGPFVDEHLLNKVDAQNEKGTKQVIKTWSRRSTILPDFIGHTFAVHDGRKHVPVFVDDSMVGHKLGEFAPTKTFKGHIKDDKKGRR
GCF_004936435.1_ASM493643v1_protein.faa
>Bifidobacterium_longum
MTRSIKKGPFVDAHLQKKVDEQNEKGTKNVIKTWSRRSMITPDFIGHTFAVHDGRKHVPVFVTEAMVGHKLGEFAPTKTFKGHVKDDKKARR
GCF_006538985.1_ASM653898v1_protein.faa
>Paenarthrobacter_aurescens
MPRSLKKGPFVDQHLFVKVARENDKGTKNVIKTWSRRSMIIPDMLGHTIAVHDGRKHIPVFVTESMVGHKLGEFAPTRTFRGHVKDDRKGKRR
GCF_000023265.1_ASM2326v1_protein

GCF_000155595.1_ASM15559v1_protein.faa
>Synechococcus_sp._PCC_7335
MSRSLKKGPFVADHLLTKIEKLNARNEKQVVKTWSRSSTIIPQFIGHTVAVHNGRQHVPVFINEQMVGHKLGEFAPTRTFRGHAKSDKKARR
GCF_000376805.1_ASM37680v1_protein.faa
>Henriciella_marina_DSM_19595
MPRSVWKGPFVDGYLLKKAENAQASERREVIKTWSRRSTIMPQFVGLNFQVHNGNKFVPVLVTEEMVGHKFGEFAPSRTYHGHGADKKAKRK
GCF_003325995.1_ASM332599v1_protein.faa
>Hyphomonas_sp._CACIAM_19H1
MSRSVWKGPFVDGYLLKKAEDARSSEKRAVIKTWSRRSTILPQFVGLNFQVHNGNKFIPVTVTDEMVGHKFGEFAPTRTYYGHGADKKAKRK
GCF_007827425.1_ASM782742v1_protein_RM_S5.faa
>Azospirillum_brasilense
MARSVWKGPFVDGYLLKKADKSRASGRNEIIKIWSRRSTILPQFVGLTFGVYNGHKFLPVLVTENMIGHKFGEFAPTRTFYGHAADKKAKRK
GCF_012911975.1_ASM1291197v1_protein_EDIT_ADD_S3.faa
>Nodosilinea_sp._P-1105
MSRSLKKGPFVADHLLSKIEALNTKGDKQVIKTWSRASTILPQMIGHTIAVHNGRQHVPVYITEQMVGHKLGEFAPTRTFRGHAKSDKKARR
GCF_000195085.1_ASM19508v1_protein.faa
>Candidatus_Pelagibacter_sp._IMCC9063
MARSIWKGPFVHPSLIKKVEKQKQTDSKRPIKTWSRNSTIIPEFIGQKFEIHNGKKFIPITISEEMIGHKLGEFSPTRQFPGHTASTSKAAPAKAVKK
GCF_003

In [33]:
checkGenome('GCF_000317635.1_ASM31763v1_protein.faa')

50S ribosomal protein L1 [
>WP_015227252.1 50S ribosomal protein L1 [Halothece sp. PCC 7418]
 MPKQPSRRMRELLEKVEDRPYEPLSALELLKETATAKFEETAEAHVRLGIDPKYSDQQIRTTVTFPKGTGQSVRVAVITRGEKVNEATEAGAEVVGSEELIQEIQNGRMDFDVVMATPDMMPQVAKLGRVLGPKGLMPSPKGGTVTNELESAIQEFKAGKQEFRADRTGIVHVMFGKVSFSAEDLLANLKALQETLDRNRPSGAKGKYWRSIYVSASMGPAIEVDIAGLRDYKLQ
1
50S ribosomal protein L2 [
>WP_015227129.1 50S ribosomal protein L2 [Halothece sp. PCC 7418]
 MSIRSYRPYTPGTRQATVSDFAEITRSKPEKSLTKYKHRKKGRNNRGVITSRRRGGGHKRRYRQIDFRRNKHNIPAEVISVEYDPNRNARISLVQYEDGEKRYILWPADLKVGDSIISGDDAPFEVGNAMPLSRIPLGTEVHNIELMAGKGGQIVRAAGTAAQVVAREKGYVTLKLPSREVRMVREECLATIGRVGNAEHRNLTLGKAGRARHLGRRPKVRGSVMNPVDHPHGGGEGRAPIGRPSPLTPWGKPTLGAKTRKAKKNSDKLIVRRRK
1
50S ribosomal protein L3 [
>WP_041596289.1 50S ribosomal protein L3 [Halothece sp. PCC 7418]
 MSIGLMGRKLGMTQIFDQEEGIAIPVTVVEVGPCIVTQIKTPSTDGYSAVQLGYKTVKEKALTKPELGHLKKVNPENTPSLRHLKEYRVDDPESYQLGQSLNADLFSEDQLVDVRGTSIGRGFAGYQKRHGFARGPMSHGSKNHRLPGSIGPGTTPGRVYPGSRMAGRMGGKTVTTRKLKVVRVDSEKNVLLIKGTIPGKPG

In [47]:
### AUTO create files
#All in one -- one shot, iterate over genome list, grab fasta, rewrite, print to screen
#Manually build files.
#Look for all Large subunits in all Taxa, print those that don't have 15.
#There's got to be a better way to do this!!!

def buildFasta(protein):
    sys.stdout = open(protein+".fasta", "w") #Output file
    for index,row in taxa_df.iterrows():
        for header, seq in fasta_iter(index):
            if header.find(protein) != -1:
                new_header = taxa_df.loc[index][1]
                new_header =  new_header.replace(' ', '_')
                print('>'+new_header+'\n'+seq) #Turn this on to get specific.
            else:
                continue
    sys.stdout.close()

In [48]:
!pwd

/nobackup1c/users/payette/ABGDtree5/sequences


In [ ]:
#Example build one fasta
buildFasta('50S ribosomal protein L1 [')

In [48]:
!cd ../

In [141]:
!pwd

/nobackup1c/users/payette/ABGDtree5/sequences


In [52]:
os.chdir('/')

In [53]:
!pwd

/


In [ ]:
# Warning these loops take a long time and you may need to restart your python kernel after they complete.

In [ ]:
for each in large_subunits:
    buildFasta(each)

In [ ]:
for each in small_subunits:
    buildFasta(each)

In [ ]:
# May need to restart kernel!

In [145]:
#Note that files will need to be renamed, since proteins are listed as follows
print(large_subunits)

['50S ribosomal protein L1 [', '50S ribosomal protein L2 [', '50S ribosomal protein L3 [', '50S ribosomal protein L4 [', '50S ribosomal protein L5 [', '50S ribosomal protein L6 [', '50S ribosomal protein L10 [', '50S ribosomal protein L13 [', '50S ribosomal protein L14 [', '50S ribosomal protein L15 [', '50S ribosomal protein L18 [', '50S ribosomal protein L22 [', '50S ribosomal protein L23 [', '50S ribosomal protein L24 [', '50S ribosomal protein L29 [']


In [ ]:
#MANUAL RENAMING OF FILES need to occur for the code below to work

In [5]:
#TO DO MAKE A PROTEIN DF so I can call name and string search easily.
proteins = ['L1','L2','L3','L4','L5','L6',
            'L10','L13','L14','L15','L18','L22',
            'L23','L24','L29','S2','S3','S4',
            'S5','S7','S8','S9','S10','S11','S12',
            'S13','S14','S15','S17','S19']

In [6]:
def runAlign(protein):
    aln_output = open('/home/payette/work/ABGDtree5/sequences/'+protein+'.aln', 'w')
    print(
        subprocess.call(['/home/payette/mafft/bin/mafft', '--reorder', '--auto',
                         '/home/payette/work/ABGDtree5/sequences/'+protein+'.fasta'],
                    stdout=aln_output))
    aln_output.close()

In [7]:
# Open an existing fasta and manipulate 
def writeFasta(protein):
    sys.stdout = open(protein+".fasta", "w") #Output file
    fasta_nam = open(protein)                #Input txt file with protein
    fasta_s = (x[1] for x in groupby(fasta_nam, lambda line: line[0] == ">"))
    for header in fasta_s:                          
        headerStr = header.__next__()
        seq = "".join(s.strip() for s in fasta_s.__next__())
        headerStr = headerStr.replace(' ', '_')
        headerStr = re.search('(?<=\[).*?(?=\])',headerStr).group()
        print('>'+headerStr+'\n'+seq)
    sys.stdout.close()
    fasta_nam.close()

In [8]:
### RUN ALIGNMENT ###

In [9]:
runAlign('L1')

0


In [10]:
#

In [11]:
runAlign('L2')

0


In [12]:
runAlign('L3')

0


In [13]:
runAlign('L4')

0


In [14]:
runAlign('L5')

0


In [15]:
runAlign('L6')

0


In [16]:
runAlign('L10')

0


In [17]:
runAlign('L13')

0


In [18]:
runAlign('L14')

0


In [19]:
runAlign('L15')

0


In [20]:
runAlign('L18')

0


In [21]:
runAlign('L22')

0


In [22]:
runAlign('L23')

0


In [23]:
runAlign('L24')

0


In [24]:
runAlign('L29')

0


In [25]:
runAlign('S2')

0


In [26]:
runAlign('S3')

0


In [27]:
runAlign('S4')

0


In [28]:
runAlign('S5')

0


In [29]:
runAlign('S7')

0


In [30]:
runAlign('S8')

0


In [31]:
runAlign('S9')

0


In [32]:
runAlign('S10')

0


In [33]:
runAlign('S11')

0


In [34]:
runAlign('S12')

0


In [35]:
runAlign('S13')

0


In [36]:
runAlign('S14')

0


In [37]:
runAlign('S15')

0


In [38]:
runAlign('S17')

0


In [39]:
runAlign('S19')

0


In [87]:
### INDIVIDUAL ALIGNMENT FINISHED!

In [75]:
from Bio import SeqIO
import sys
import re
import os

In [85]:
os.chdir('/home/payette/work/ABGDtree5/sequences')
!pwd

/nobackup1c/users/payette/ABGDtree5/sequences


In [86]:
!ls -lR *.aln | wc -l

30


In [87]:
from Bio import AlignIO, SeqIO, Align, Alphabet
import pandas as pd
import os, re, sys
from copy import deepcopy

aln_extension = '.aln'

In [88]:
!pwd

/nobackup1c/users/payette/ABGDtree5/sequences


In [89]:
aln_alphabet = Alphabet.Gapped(Alphabet.IUPAC.ambiguous_dna)
genomes    = {}
for aln in os.listdir('.'):
    #
    # check if files have the desired extension
    if not aln.endswith(aln_extension):
        continue
    alignment    = AlignIO.read(aln, 'fasta')
    genomes[aln] = set()
    print(alignment)
    for entry in alignment:
        genome = entry.name
        #print(genome)
        if genome in genomes[aln]:
            #print ('\t**Error, duplicated genome in %s: %s' %(aln, genome))
            sys.exit('\t**Error, duplicated genome in %s: %s' %(aln, genome))
            break
        genomes[aln].add(genome)

SingleLetterAlphabet() alignment with 306 rows and 290 columns
MARYIGPK--CKLARREG-TDLFLK---SGARPLDSKC-KA-E-...K-- Simiduia_agarivorans_SA1_=_DSM_21679
MARYIGPT--CKLSRREG-TDLFLK---SGARALDSKC-KI-E-...K-- Cellvibrio_mixtus
MARYIGPK--CKLSRREG-TDLQLK---SGVRPHDSKC-RA-E-...K-- Microbulbifer_agarilyticus
MARYIGPK--CKLARREG-TDLFLK---SGVRSLESKC-KP-E-...K-- Congregibacter_litoralis_KT71
MARYIGPK--CKLARREG-TDLFLK---SGVRALESKC-KP-E-...K-- Haliea_salexigens_DSM_19537
MARYIGPK--CKLSRREG-TDLFLK---SGVRALESKC-KA-E-...K-- Endozoicomonas_elysicola_DSM_22380
MARYIGPK--CKLSRREG-TDLFLK---SGARALDSKC-KA-E-...K-- Pseudomonas_aeruginosa_148
MARYLGPT--CKLSRREG-TDLFLK---SGVRALDSKC-KA-E-...K-- Amphritea_japonica_ATCC_BAA-1530
MARYIGPT--CKLSRREG-TDLQLK---SGSRALESKC-KV-E-...K-- Marinomonas_arctica
MARYIGPK--CKLSRREG-TDLFLK---SGVTPFEKKC-KS-E-...K-- Halomonas_alkaliantarctica
MARYLGPK--LKLSRREG-TDLFLK---SGVRAIDSKC-KI-D-...K-- Alteromonas_macleodii_ATCC_27126
MARYLGPK--LKLSRREG-TDLFLK---SGVRAIDSKC-KI-E-...K-- Idiomarina

In [90]:
genome_union  = set.union(*genomes.values())
missing_genes = {} # just to keep track of the number of missing marker genes in each genome
concatenation = {}
for genome in genome_union:
    missing_genes[genome]             = 0
    concatenation[genome]             = Align.SeqRecord( Align.Seq('', aln_alphabet) )
    concatenation[genome].name        = genome
    concatenation[genome].id          = genome
    concatenation[genome].description = genome

In [91]:
#
# fill the handles with the marker sequences from each genome
total_genes      = 30 # keep track of the number of genes added to the concatenation
current_position = 1
partitions       = open('ABGDtree5_concatenated_partitions', 'w')
for aln in os.listdir('.'):
    if not aln.endswith(aln_extension):
        continue
    
    print(aln.replace(aln_extension, ''))
    tmp_aln      = AlignIO.read(aln, 'fasta' )
    aln_length   = tmp_aln.get_alignment_length() # get the expected size of the alignment so you can 
                                                  #   compare if all have the same size
    total_genes += aln_length

    for entry in tmp_aln:
        # if this alignment has a different size from the rest, something is reaaaaaly wrong!
        if len(entry) != aln_length:
            sys.exit('\t**Error, block "%s" has a different length than the rest of the MSA: %s' % 
                     (entry.name, aln))

        genome = entry.name
        concatenation[genome] += deepcopy(entry.seq)

    partitions.write('LG, %s = %i-%i\n' %
                     (aln.replace(aln_extension, ''), current_position, current_position+aln_length-1) )
    current_position += aln_length

    #
    # add gaps for those genomes missing this gene (same size as the expected alignment)
    for genome in genome_union.difference(genomes[aln]):
        concatenation[genome] += Align.Seq( '-' * aln_length, aln_alphabet )
        missing_genes[genome] += aln_length
partitions.close()

S4
S12
S5
L22
S15
S9
L4
L2
S3
S17
S2
L18
S8
S14
S11
L10
L6
L15
L14
L3
L29
L13
S7
L5
S10
S19
S13
L24
L1
L23


In [92]:
# Don't use!
# remove genomes missing more than 20% of the marker genes
#counter = 0
#for genome, num_missing_genes in missing_genes.items():
#    if num_missing_genes/total_genes > 0.8:
#        print('\t\t**%s: missing %.2f%% from concatenated alignment!' %(genome, (num_missing_genes/total_genes)*100))
#        counter += 1
#
#print('%i genomes missing more than 10%%' %counter)

In [93]:
AlignIO.write(Align.MultipleSeqAlignment(concatenation.values() ), 'ABGDtree5.aln', 'fasta')

1

In [94]:
####
# Had to manually adjust .aln file for duplicate names in Buchnera_a taxa!

In [95]:
### MANUAL NAME CONVERSION KEY

# >Buchnera_aphidicola_str._APS_(Acyrthosiphon_pisum)
# >Buchnera_a_str_Acyrthosiphon_pisum

# >Buchnera_aphidicola_str._Bp_(Baizongia_pistaciae)
# >Buchnera_a_str_Baizongia_pistaciae

In [96]:
from Bio import SeqIO
import sys
import re
import os
#!conda install -c bioconda iqtree

In [97]:
aln_name   = 'ABGDtree5.aln'
fasta      = open(aln_name).read()
taxa_names = re.findall('^>(\S+?)[\s|]', fasta, re.M)
for count, name in enumerate(taxa_names):
    fasta = re.sub('^>%s' % name, '>taxa_%i_ ' % count, fasta, flags=re.M)

In [98]:
out = open('ABGDtree5.fas', 'w')
out.write(fasta)
out.close()

In [99]:
SeqIO.convert('ABGDtree5.fas', 'fasta', 'ABGDtree5.phy', 'phylip')
phylip = open('ABGDtree5.phy').read()
for count, name in enumerate(taxa_names):
    phylip = re.sub('^taxa_%i_\s+' % count, '%s  ' % name, phylip, flags=re.M)

In [100]:
out = open('%s.phy' % aln_name, 'w')
out.write(phylip)
out.close()

In [110]:
#get iq tree to work on server ... for now do it manually...

In [101]:
# !/nobackup1c/users/payette/iqtree-2.1.2-Linux/bin/iqtree2 -nt AUTO -ntmax 10 -s ABGDtree5.aln.phy -m MFP -mrate E,I,G4,I+G4,R4 -madd LG4X,C20 -keep-ident -safe -mem 15G -bb 1000 -alrt 1000

In [ ]:
# END ############